In [1]:
import os

# Check current directory
print("Current directory:", os.getcwd())

# Change to your code directory (replace with your actual path)
os.chdir('/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/full_frozen_geneformer/Geneformer/geneformer')

# Verify the change
print("New directory:", os.getcwd())

Current directory: /scratchdata1/users/a1841503/Geneformer/Jupyter
New directory: /scratchdata1/groups/phoenix-hpc-mangiola_laboratory/haroon/full_frozen_geneformer/Geneformer/geneformer


In [2]:
import os
import sys

# Change to the repository root
os.chdir("/scratchdata1/groups/phoenix-hpc-mangiola_laboratory/haroon/full_frozen_geneformer/Geneformer")

# Add current directory to Python path
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

In [3]:
# !jupyter kernelspec list


In [4]:
# import transformers
# transformers.__version__

In [ ]:
from geneformer.finetuner import FineTuner

In [ ]:
from geneformer.finetuner_utils import get_train_valid_test_splits

In [ ]:
import json
import pickle


In [ ]:
!pwd

In [ ]:
task_config = {
        "tasks": {
        "disease_classification": [
            # "genecorpus_heart_disease", 
            # "cellnexus_blood_disease", 
            "cellnexus_covid_disease"
            ],
        # "dosage_sensitivity": ["genecorpus_dosage_sensitivity"],
        "cell_type_classification": ["cellnexus_cell_types"]
        },
        "aggregation_levels": [
            "singlecell",
            "metacell_2", 
            "metacell_4", 
            "metacell_8", 
            "metacell_16", 
            # "metacell_32", 
            # "metacell_64", 
            # "metacell_128"
            ]
    }

# aggregation_level="metacell_32"
# task="dosage_sensitivity"
# dataset="genecorpus_dosage_sensitivity"
model_version="V1"
base_dir= "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer"
model_variant="30M"
crossval_splits = 5 # 1 or 5
freeze_num_encoder_layers=6
freeze_entire_model=True

    # Loop over tasks
for task, datasets in task_config["tasks"].items():
    # Loop over each dataset associated with the task
    for dataset in datasets:
        # Loop over aggregation levels
        for aggregation_level in task_config["aggregation_levels"]:
            print(f"Running task={task}, dataset={dataset}, aggregation_level={aggregation_level}")

            if task not in ["dosage_sensitivity", "cell_type_classification"]:
                crossval_split_metrics = {}

                for crossval_split in range(1, crossval_splits + 1):
                    print(f"Running cross-validation split {crossval_split} of {crossval_splits} for task={task}, dataset={dataset}, aggregation_level={aggregation_level}")

                    



                    training_args = {
                    "num_train_epochs": 10,
                    "learning_rate": 0.000804,
                    "lr_scheduler_type": "polynomial",
                    "warmup_steps": 1812,
                    "weight_decay":0.258828,
                    "per_device_train_batch_size": 128,
                    "seed": 73,
                    "evaluation_strategy":"epoch",        # Evaluate every epoch
                    "save_strategy":"epoch",              # Save checkpoint every epoch
                    "metric_for_best_model":"eval_loss",  # Metric to determine "best" model # Doc: https://huggingface.co/transformers/v3.5.1/main_classes/trainer.html#:~:text=after%20each%20evaluation.-,metric_for_best_model,-(str%2C
                    "greater_is_better":False,            # For loss, lower is better
                    "load_best_model_at_end":True,        # KEY: Load best model at the end
                    "save_total_limit":3,                 # Keep only 3 best checkpoints
                    # "logging_dir": os.path.normpath("D:/geneformer_finetuning/trained_cell_classification_models/disease_classification/genecorpus_heart_disease/30M_metacell_8/250623_geneformer_cellClassifier_genecorpus_heart_disease_test/ksplit1/runs"),
                    }
                    

                    input_data_file, cell_state_dict, filter_data_dict, train_test_id_split_dict, train_valid_id_split_dict = get_train_valid_test_splits(
                        TASK=task,
                        DATASET=dataset,
                        MODEL_VARIANT=model_variant,
                        DATASET_PATH = os.path.join(base_dir, "datasets", task, dataset),
                        CROSSVAL_SPLITS=crossval_splits,
                    )



                    finetuner = FineTuner(base_dir=base_dir,
                        aggregation_level=aggregation_level,
                        model_variant=model_variant,
                        task=task,
                        dataset=dataset,
                        crossval_splits=crossval_splits,)
                    

                    
                    output_prefix = "test" if crossval_splits == 1 else "ksplit" + str(crossval_split)

                    model_dir = os.path.join(finetuner.output_dir, f"geneformer_cellClassifier_{output_prefix}")
                    existing_metrics_path = os.path.join(model_dir, f"{output_prefix}_eval_metrics_dict.pkl")
                    
                    if os.path.exists(existing_metrics_path):
                        print(f"Loading existing metrics for cross-validation split {crossval_split} from {existing_metrics_path}")
                        # FIXED: Variable name consistency
                        with open(existing_metrics_path, "rb") as f:
                            all_metrics = pickle.load(f)

                    else:
        
                        all_metrics = finetuner.finetune_model(
                            training_args=training_args, 
                            cell_state_dict = cell_state_dict, 
                            filter_data_dict = filter_data_dict, 
                            input_data_file = input_data_file, 
                            output_prefix = output_prefix, 
                            train_test_id_split_dict = train_test_id_split_dict[str(crossval_split)] if train_test_id_split_dict is not None else None, 
                            train_valid_id_split_dict = train_valid_id_split_dict[str(crossval_split)] if train_valid_id_split_dict is not None else None, 
                            num_crossval_splits= 1 if task not in ["dosage_sensitivity", "cell_type_classification"] else crossval_splits,
                            freeze_num_encoder_layers=freeze_num_encoder_layers,
                            freeze_entire_model=freeze_entire_model,
                            split_sizes={"train": 0.8, "valid": 0.2, "test": 0.0}
                            )
                    
                    crossval_split_metrics[str(crossval_split)] = all_metrics

                metrics_path = os.path.join(finetuner.output_dir, "metrics.pkl")
                with open(metrics_path, "wb") as f:
                    pickle.dump(crossval_split_metrics, f)

                print(f"✅ Metrics saved to: {metrics_path}")

            elif task in ["dosage_sensitivity", "cell_type_classification"]:

                print(f"Running cross-validation for task={task}, dataset={dataset}, aggregation_level={aggregation_level}")

                



                training_args = {
                "num_train_epochs": 10,
                "learning_rate": 0.000804,
                "lr_scheduler_type": "polynomial",
                "warmup_steps": 1812,
                "weight_decay":0.258828,
                "per_device_train_batch_size": 128,
                "seed": 73,
                "evaluation_strategy":"epoch",        # Evaluate every epoch
                "save_strategy":"epoch",              # Save checkpoint every epoch
                "metric_for_best_model":"eval_loss",  # Metric to determine "best" model # Doc: https://huggingface.co/transformers/v3.5.1/main_classes/trainer.html#:~:text=after%20each%20evaluation.-,metric_for_best_model,-(str%2C
                "greater_is_better":False,            # For loss, lower is better
                "load_best_model_at_end":True,        # KEY: Load best model at the end
                "save_total_limit":3,                 # Keep only 3 best checkpoints
                # "logging_dir": os.path.normpath("D:/geneformer_finetuning/trained_cell_classification_models/disease_classification/genecorpus_heart_disease/30M_metacell_8/250623_geneformer_cellClassifier_genecorpus_heart_disease_test/ksplit1/runs"),
                }
                

                input_data_file, cell_state_dict, filter_data_dict, train_test_id_split_dict, train_valid_id_split_dict = get_train_valid_test_splits(
                    TASK=task,
                    DATASET=dataset,
                    MODEL_VARIANT=model_variant,
                    DATASET_PATH = os.path.join(base_dir, "datasets", task, dataset)
,
                    CROSSVAL_SPLITS=crossval_splits,
                )



                finetuner = FineTuner(base_dir=base_dir,
                    aggregation_level=aggregation_level,
                    model_variant=model_variant,
                    task=task,
                    dataset=dataset,
                    crossval_splits=crossval_splits,)
                
                output_prefix = "5fold" if crossval_splits == 5 else "test"

                metrics_path = os.path.join(finetuner.output_dir, "metrics.pkl")
                if os.path.exists(metrics_path):
                    print(f"Loading existing metrics for cross-validation splits from {metrics_path}")
                    with open(metrics_path, "rb") as f:
                        all_metrics = pickle.load(f)

                else:
                    print(f"Running finetuning for task={task}, dataset={dataset}, aggregation_level={aggregation_level}")
                
                
    
                    all_metrics = finetuner.finetune_model(
                        training_args=training_args, 
                        cell_state_dict = cell_state_dict, 
                        filter_data_dict = filter_data_dict, 
                        input_data_file = input_data_file, 
                        output_prefix = output_prefix, 
                        train_test_id_split_dict = train_test_id_split_dict[str(crossval_split)] if train_test_id_split_dict is not None else None, 
                        train_valid_id_split_dict = train_valid_id_split_dict[str(crossval_split)] if train_valid_id_split_dict is not None else None, 
                        num_crossval_splits= 1 if task not in ["dosage_sensitivity", "cell_type_classification"] else crossval_splits,
                        freeze_num_encoder_layers=freeze_num_encoder_layers,
                        freeze_entire_model=freeze_entire_model,
                        split_sizes={"train": 0.8, "valid": 0.2, "test": 0.0}
                        )
                    

                    with open(metrics_path, "wb") as f:
                        pickle.dump(all_metrics, f)

                    print(f"✅ Metrics saved to: {metrics_path}")

In [ ]:
# !pip install transformers==4.49.0

In [ ]:
with open(metrics_path, "wb") as f:
    pickle.dump(all_metrics, f)

In [ ]:
all_metrics

In [ ]:
with open("/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/5fold_cv_trained_cell_classification_models/disease_classification/genecorpus_heart_disease/30M_singlecell/metrics.pkl", "rb") as f:
    crossval_split_metrics = pickle.load(f)

In [ ]:
crossval_split_metrics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from typing import Dict, Any, Optional, Tuple

def plot_cv_metrics_and_confusion_matrices(crossval_split_metrics: Dict[str, Dict[str, Any]], 
                                          normalize_cm: bool = False,
                                          figsize: tuple = (16, 12),
                                          y_min: Optional[float] = None,
                                          y_max: Optional[float] = None,
                                          row_spacing: float = 0.3,
                                          show_annotations: bool = True,
                                          save_path: Optional[str] = None) -> None:
    """
    Plot cross-validation metrics (accuracy and F1) as line plots and confusion matrices as subplots.
    
    Parameters:
    -----------
    crossval_split_metrics : dict
        Dictionary containing metrics for each fold
    normalize_cm : bool, default=False
        Whether to normalize confusion matrices
    figsize : tuple, default=(16, 12)
        Figure size for the plot
    y_min : float, optional
        Minimum y-axis value for metrics plots
    y_max : float, optional
        Maximum y-axis value for metrics plots
    row_spacing : float, default=0.3
        Spacing between rows of subplots
    show_annotations : bool, default=True
        Whether to show value annotations on line plots
    save_path : str, optional
        Path to save the figure
    """
    
    # Extract metrics for line plots
    folds = []
    accuracies = []
    f1_scores = []
    
    for fold, metrics in crossval_split_metrics.items():
        folds.append(int(fold))
        accuracies.append(metrics['acc'][0] if isinstance(metrics['acc'], list) else metrics['acc'])
        f1_scores.append(metrics['macro_f1'][0] if isinstance(metrics['macro_f1'], list) else metrics['macro_f1'])
    
    # Sort by fold number
    sorted_data = sorted(zip(folds, accuracies, f1_scores))
    folds, accuracies, f1_scores = zip(*sorted_data)
    
    # Create subplots: 3 rows, 3 columns
    # Row 1: Line plots (col 1-2), Row 2-3: Confusion matrices (2x3 grid)
    fig = plt.figure(figsize=figsize)
    
    # Create GridSpec for custom layout with adjustable spacing
    from matplotlib.gridspec import GridSpec
    gs = GridSpec(3, 3, figure=fig, height_ratios=[1, 1, 1], hspace=row_spacing, wspace=0.3)
    
    # Plot 1: Accuracy line plot
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(folds, accuracies, marker='o', linewidth=2, markersize=8, color='blue', label='Accuracy')
    ax1.set_xlabel('Fold Number')
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Accuracy Across Folds', fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(folds)
    
    # Set y-axis limits if specified
    if y_min is not None or y_max is not None:
        ax1.set_ylim(y_min, y_max)
    
    # Add value labels on points
    if show_annotations:
        for i, (fold, acc) in enumerate(zip(folds, accuracies)):
            ax1.annotate(f'{acc:.3f}', (fold, acc), textcoords="offset points", 
                        xytext=(0,10), ha='center', fontsize=9)
    
    # Plot 2: F1 Score line plot
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(folds, f1_scores, marker='s', linewidth=2, markersize=8, color='red', label='Macro F1')
    ax2.set_xlabel('Fold Number')
    ax2.set_ylabel('Macro F1 Score')
    ax2.set_title('Macro F1 Score Across Folds', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(folds)
    
    # Set y-axis limits if specified
    if y_min is not None or y_max is not None:
        ax2.set_ylim(y_min, y_max)
    
    # Add value labels on points
    if show_annotations:
        for i, (fold, f1) in enumerate(zip(folds, f1_scores)):
            ax2.annotate(f'{f1:.3f}', (fold, f1), textcoords="offset points", 
                        xytext=(0,10), ha='center', fontsize=9)
    
    # Plot 3: Combined metrics
    ax3 = fig.add_subplot(gs[0, 2])
    line1 = ax3.plot(folds, accuracies, marker='o', linewidth=2, markersize=6, color='blue', label='Accuracy')
    line2 = ax3.plot(folds, f1_scores, marker='s', linewidth=2, markersize=6, color='red', label='Macro F1')
    ax3.set_xlabel('Fold Number')
    ax3.set_ylabel('Score')
    ax3.set_title('Combined Metrics', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.set_xticks(folds)
    ax3.legend()
    
    # Set y-axis limits if specified
    if y_min is not None or y_max is not None:
        ax3.set_ylim(y_min, y_max)
    
    # Add annotations for combined metrics
    if show_annotations:
        for i, (fold, acc, f1) in enumerate(zip(folds, accuracies, f1_scores)):
            ax3.annotate(f'{acc:.3f}', (fold, acc), textcoords="offset points", 
                        xytext=(0,10), ha='center', fontsize=8, color='blue')
            ax3.annotate(f'{f1:.3f}', (fold, f1), textcoords="offset points", 
                        xytext=(0,-15), ha='center', fontsize=8, color='red')
    
    # Plot confusion matrices in 2x3 grid
    cm_positions = [(1, 0), (1, 1), (1, 2), (2, 0), (2, 1)]
    
    for i, (fold, metrics) in enumerate(sorted(crossval_split_metrics.items(), key=lambda x: int(x[0]))):
        if i >= 5:  # Only plot first 5 folds
            break
            
        row, col = cm_positions[i]
        ax = fig.add_subplot(gs[row, col])
        
        # Extract confusion matrix
        conf_matrix = metrics['conf_matrix']
        
        # Convert to numpy array if needed
        if hasattr(conf_matrix, 'values'):
            cm_array = conf_matrix.values
            class_names = conf_matrix.index.tolist()
        else:
            # Assume it's already a DataFrame or array
            cm_array = np.array(conf_matrix)
            class_names = ['nf', 'hcm', 'dcm']  # Default class names
        
        # Normalize if requested
        if normalize_cm:
            cm_array = cm_array.astype('float') / cm_array.sum(axis=1)[:, np.newaxis]
            fmt = '.2f'
            title_suffix = ' (Normalized)'
        else:
            fmt = '.0f'
            title_suffix = ''
        
        # Plot heatmap
        sns.heatmap(cm_array, annot=True, fmt=fmt, cmap='Blues', 
                   xticklabels=class_names, yticklabels=class_names, ax=ax)
        ax.set_title(f'Fold {fold} Confusion Matrix{title_suffix}', fontweight='bold')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('Actual')
    
    # Add overall title
    fig.suptitle('Cross-Validation Results: Metrics and Confusion Matrices', 
                fontsize=16, fontweight='bold', y=0.98)
    
    # Add summary statistics as text
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    summary_text = f'Summary Statistics:\n'
    summary_text += f'Accuracy: {mean_acc:.3f} ± {std_acc:.3f}\n'
    summary_text += f'Macro F1: {mean_f1:.3f} ± {std_f1:.3f}'
    
    fig.text(0.98, 0.02, summary_text, fontsize=10, ha='right', va='bottom',
            bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Plot saved to: {save_path}")
    
    plt.show()


def compute_average_metrics(crossval_split_metrics: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    """
    Compute average metrics across all cross-validation folds.
    
    Parameters:
    -----------
    crossval_split_metrics : dict
        Dictionary containing metrics for each fold
        
    Returns:
    --------
    dict
        Dictionary with averaged metrics: {'sum_conf_matrix', 'avg_macro_f1', 'avg_acc'}
    """
    
    # Extract individual metrics
    accuracies = []
    f1_scores = []
    confusion_matrices = []
    
    for fold, metrics in crossval_split_metrics.items():
        # Extract accuracy
        acc = metrics['acc'][0] if isinstance(metrics['acc'], list) else metrics['acc']
        accuracies.append(acc)
        
        # Extract F1 score
        f1 = metrics['macro_f1'][0] if isinstance(metrics['macro_f1'], list) else metrics['macro_f1']
        f1_scores.append(f1)
        
        # Extract confusion matrix
        conf_matrix = metrics['conf_matrix']
        if hasattr(conf_matrix, 'values'):
            cm_array = conf_matrix.values
        else:
            cm_array = np.array(conf_matrix)
        confusion_matrices.append(cm_array)
    
    # Compute averages for metrics and sum for confusion matrix
    avg_accuracy = np.mean(accuracies)
    avg_f1 = np.mean(f1_scores)
    sum_confusion_matrix = np.sum(confusion_matrices, axis=0)  # Sum instead of mean
    
    # Create result dictionary
    avg_metrics = {
        'avg_acc': avg_accuracy,
        'avg_macro_f1': avg_f1,
        'sum_conf_matrix': sum_confusion_matrix  # Updated key name to reflect it's a sum
    }
    
    print(f"📊 Average Metrics Computed:")
    print(f"   Average Accuracy: {avg_accuracy:.4f}")
    print(f"   Average Macro F1: {avg_f1:.4f}")
    print(f"   Summed Confusion Matrix shape: {sum_confusion_matrix.shape}")
    
    return avg_metrics


def plot_average_metrics(avg_metrics: Dict[str, Any], 
                        class_names: list = ['nf', 'hcm', 'dcm'],
                        normalize_cm: bool = False,
                        figsize: tuple = (12, 5),
                        y_min: Optional[float] = None,
                        y_max: Optional[float] = None,
                        show_annotations: bool = True,
                        bar_width: float = 0.6,
                        save_path: Optional[str] = None) -> None:
    """
    Plot average metrics: bar chart for accuracy/F1 and heatmap for confusion matrix.
    
    Parameters:
    -----------
    avg_metrics : dict
        Dictionary containing averaged metrics
    class_names : list, default=['nf', 'hcm', 'dcm']
        Names of the classes for confusion matrix labels
    normalize_cm : bool, default=False
        Whether to normalize the confusion matrix
    figsize : tuple, default=(12, 5)
        Figure size for the plot
    y_min : float, optional
        Minimum y-axis value for bar chart
    y_max : float, optional
        Maximum y-axis value for bar chart
    show_annotations : bool, default=True
        Whether to show value annotations on bars
    bar_width : float, default=0.6
        Width of the bars in the bar chart
    save_path : str, optional
        Path to save the figure
    """
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Plot 1: Bar chart of average metrics
    metrics_names = ['Accuracy', 'Macro F1']
    metrics_values = [avg_metrics['avg_acc'], avg_metrics['avg_macro_f1']]
    colors = ['skyblue', 'lightcoral']
    
    bars = ax1.bar(metrics_names, metrics_values, color=colors, alpha=0.8, 
                   edgecolor='black', width=bar_width)
    ax1.set_ylabel('Score')
    ax1.set_title('Average Cross-Validation Metrics', fontweight='bold')
    
    # Set y-axis limits if specified
    if y_min is not None or y_max is not None:
        ax1.set_ylim(y_min, y_max)
    else:
        ax1.set_ylim(0, 1)
    
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    if show_annotations:
        for bar, value in zip(bars, metrics_values):
            height = bar.get_height()
            ax1.annotate(f'{value:.3f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3), textcoords="offset points", ha='center', va='bottom',
                        fontweight='bold')
    
    # Plot 2: Average confusion matrix
    cm_array = avg_metrics['sum_conf_matrix']  # Updated key name
    
    # Normalize if requested
    if normalize_cm:
        cm_array = cm_array.astype('float') / cm_array.sum(axis=1)[:, np.newaxis]
        fmt = '.3f'
        title_suffix = ' (Normalized)'
        cmap = 'Blues'
    else:
        fmt = '.0f'
        title_suffix = ''
        cmap = 'Blues'
    
    # Plot heatmap
    sns.heatmap(cm_array, annot=True, fmt=fmt, cmap=cmap, 
               xticklabels=class_names, yticklabels=class_names, ax=ax2)
    ax2.set_title(f'Summed Confusion Matrix{title_suffix}', fontweight='bold')  # Updated title
    ax2.set_xlabel('Predicted')
    ax2.set_ylabel('Actual')
    
    # Add summary statistics
    summary_text = f'Cross-Validation Summary:\n'
    summary_text += f'Accuracy: {avg_metrics["avg_acc"]:.4f}\n'
    summary_text += f'Macro F1: {avg_metrics["avg_macro_f1"]:.4f}'
    
    fig.text(0.02, 0.98, summary_text, fontsize=10, ha='left', va='top',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
            transform=fig.transFigure)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Average metrics plot saved to: {save_path}")
    
    plt.show()


def comprehensive_cv_analysis(crossval_split_metrics: Dict[str, Dict[str, Any]],
                             class_names: list = ['nf', 'hcm', 'dcm'],
                             normalize_cm: bool = False,
                             y_min: Optional[float] = None,
                             y_max: Optional[float] = None,
                             row_spacing: float = 0.3,
                             show_annotations: bool = True,
                             bar_width: float = 0.6,
                             save_individual: Optional[str] = None,
                             save_average: Optional[str] = None) -> Dict[str, Any]:
    """
    Comprehensive analysis: plot individual fold results, compute averages, and plot averages.
    
    Parameters:
    -----------
    crossval_split_metrics : dict
        Dictionary containing metrics for each fold
    class_names : list, default=['nf', 'hcm', 'dcm']
        Names of the classes
    normalize_cm : bool, default=False
        Whether to normalize confusion matrices
    y_min : float, optional
        Minimum y-axis value for metrics plots
    y_max : float, optional
        Maximum y-axis value for metrics plots
    row_spacing : float, default=0.3
        Spacing between rows of subplots
    show_annotations : bool, default=True
        Whether to show value annotations on plots
    bar_width : float, default=0.6
        Width of the bars in average metrics bar chart
    save_individual : str, optional
        Path to save individual folds plot
    save_average : str, optional
        Path to save average metrics plot
        
    Returns:
    --------
    dict
        Dictionary with averaged metrics
    """
    
    print("🔍 Starting comprehensive cross-validation analysis...")
    
    # Plot individual fold results
    print("\n📊 Plotting individual fold results...")
    plot_cv_metrics_and_confusion_matrices(
        crossval_split_metrics, 
        normalize_cm=normalize_cm,
        y_min=y_min,
        y_max=y_max,
        row_spacing=row_spacing,
        show_annotations=show_annotations,
        save_path=save_individual
    )
    
    # Compute average metrics
    print("\n🧮 Computing average metrics...")
    avg_metrics = compute_average_metrics(crossval_split_metrics)
    
    # Plot average results
    print("\n📈 Plotting average results...")
    plot_average_metrics(
        avg_metrics, 
        class_names=class_names, 
        normalize_cm=normalize_cm,
        y_min=y_min,
        y_max=y_max,
        show_annotations=show_annotations,
        bar_width=bar_width,
        save_path=save_average
    )
    
    # Print detailed summary
    print("\n" + "="*60)
    print("CROSS-VALIDATION SUMMARY")
    print("="*60)
    
    # Individual fold results
    folds = sorted([int(k) for k in crossval_split_metrics.keys()])
    print(f"\nIndividual Fold Results:")
    print(f"{'Fold':<6}{'Accuracy':<12}{'Macro F1':<12}")
    print("-" * 30)
    
    for fold in folds:
        fold_str = str(fold)
        acc = crossval_split_metrics[fold_str]['acc'][0] if isinstance(crossval_split_metrics[fold_str]['acc'], list) else crossval_split_metrics[fold_str]['acc']
        f1 = crossval_split_metrics[fold_str]['macro_f1'][0] if isinstance(crossval_split_metrics[fold_str]['macro_f1'], list) else crossval_split_metrics[fold_str]['macro_f1']
        print(f"{fold:<6}{acc:<12.4f}{f1:<12.4f}")
    
    # Summary statistics
    accs = [crossval_split_metrics[str(f)]['acc'][0] if isinstance(crossval_split_metrics[str(f)]['acc'], list) else crossval_split_metrics[str(f)]['acc'] for f in folds]
    f1s = [crossval_split_metrics[str(f)]['macro_f1'][0] if isinstance(crossval_split_metrics[str(f)]['macro_f1'], list) else crossval_split_metrics[str(f)]['macro_f1'] for f in folds]
    
    print(f"\nSummary Statistics:")
    print(f"Accuracy  - Mean: {np.mean(accs):.4f}, Std: {np.std(accs):.4f}, Min: {np.min(accs):.4f}, Max: {np.max(accs):.4f}")
    print(f"Macro F1  - Mean: {np.mean(f1s):.4f}, Std: {np.std(f1s):.4f}, Min: {np.min(f1s):.4f}, Max: {np.max(f1s):.4f}")
    
    print(f"\n✅ Analysis completed successfully!")
    
    return avg_metrics

In [ ]:
# Basic usage (same as before)
avg_metrics = comprehensive_cv_analysis(crossval_split_metrics,
                                        class_names = ['nf', 'hcm', 'dcm'],
                             normalize_cm = True,
                             y_min = 0.35,
                             y_max = 0.65,
                             row_spacing = 0.5,
                             show_annotations = True,
                             bar_width = 0.6,)

# With custom parameters
avg_metrics = comprehensive_cv_analysis(
    crossval_split_metrics,
    y_min=0.7,           # Set minimum y-axis value
    y_max=1.0,           # Set maximum y-axis value
    row_spacing=0.5,     # Increase spacing between rows
    show_annotations=False,  # Hide annotations
    bar_width=0.8,       # Wider bars
    normalize_cm=True
)

# # Individual function calls with parameters
# plot_cv_metrics_and_confusion_matrices(
#     crossval_split_metrics,
#     y_min=0.8,
#     y_max=1.0,
#     row_spacing=0.4,
#     show_annotations=True
# )

# plot_average_metrics(
#     avg_metrics,
#     y_min=0.85,
#     y_max=0.95,
#     bar_width=0.5,
#     show_annotations=True
# )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from typing import Dict, Any, Optional

def plot_cv_metrics_and_confusion_matrices(crossval_split_metrics: Dict[str, Dict[str, Any]], 
                                          normalize_cm: bool = False,
                                          figsize: tuple = (16, 12),
                                          save_path: Optional[str] = None) -> None:
    """
    Plot cross-validation metrics (accuracy and F1) as line plots and confusion matrices as subplots.
    
    Parameters:
    -----------
    crossval_split_metrics : dict
        Dictionary containing metrics for each fold
    normalize_cm : bool, default=False
        Whether to normalize confusion matrices
    figsize : tuple, default=(16, 12)
        Figure size for the plot
    save_path : str, optional
        Path to save the figure
    """
    
    # Extract metrics for line plots
    folds = []
    accuracies = []
    f1_scores = []
    
    for fold, metrics in crossval_split_metrics.items():
        folds.append(int(fold))
        accuracies.append(metrics['acc'][0] if isinstance(metrics['acc'], list) else metrics['acc'])
        f1_scores.append(metrics['macro_f1'][0] if isinstance(metrics['macro_f1'], list) else metrics['macro_f1'])
    
    # Sort by fold number
    sorted_data = sorted(zip(folds, accuracies, f1_scores))
    folds, accuracies, f1_scores = zip(*sorted_data)
    
    # Create subplots: 3 rows, 3 columns
    # Row 1: Line plots (col 1-2), Row 2-3: Confusion matrices (2x3 grid)
    fig = plt.figure(figsize=figsize)
    
    # Create GridSpec for custom layout
    from matplotlib.gridspec import GridSpec
    gs = GridSpec(3, 3, figure=fig, height_ratios=[1, 1, 1], hspace=0.3, wspace=0.3)
    
    # Plot 1: Accuracy line plot
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(folds, accuracies, marker='o', linewidth=2, markersize=8, color='blue', label='Accuracy')
    ax1.set_xlabel('Fold Number')
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Accuracy Across Folds', fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(folds)
    
    # Add value labels on points
    for i, (fold, acc) in enumerate(zip(folds, accuracies)):
        ax1.annotate(f'{acc:.3f}', (fold, acc), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontsize=9)
    
    # Plot 2: F1 Score line plot
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(folds, f1_scores, marker='s', linewidth=2, markersize=8, color='red', label='Macro F1')
    ax2.set_xlabel('Fold Number')
    ax2.set_ylabel('Macro F1 Score')
    ax2.set_title('Macro F1 Score Across Folds', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(folds)
    
    # Add value labels on points
    for i, (fold, f1) in enumerate(zip(folds, f1_scores)):
        ax2.annotate(f'{f1:.3f}', (fold, f1), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontsize=9)
    
    # Plot 3: Combined metrics
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.plot(folds, accuracies, marker='o', linewidth=2, markersize=6, color='blue', label='Accuracy')
    ax3.plot(folds, f1_scores, marker='s', linewidth=2, markersize=6, color='red', label='Macro F1')
    ax3.set_xlabel('Fold Number')
    ax3.set_ylabel('Score')
    ax3.set_title('Combined Metrics', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.set_xticks(folds)
    ax3.legend()
    
    # Plot confusion matrices in 2x3 grid
    cm_positions = [(1, 0), (1, 1), (1, 2), (2, 0), (2, 1)]
    
    for i, (fold, metrics) in enumerate(sorted(crossval_split_metrics.items(), key=lambda x: int(x[0]))):
        if i >= 5:  # Only plot first 5 folds
            break
            
        row, col = cm_positions[i]
        ax = fig.add_subplot(gs[row, col])
        
        # Extract confusion matrix
        conf_matrix = metrics['conf_matrix']
        
        # Convert to numpy array if needed
        if hasattr(conf_matrix, 'values'):
            cm_array = conf_matrix.values
            class_names = conf_matrix.index.tolist()
        else:
            # Assume it's already a DataFrame or array
            cm_array = np.array(conf_matrix)
            class_names = ['nf', 'hcm', 'dcm']  # Default class names
        
        # Normalize if requested
        if normalize_cm:
            cm_array = cm_array.astype('float') / cm_array.sum(axis=1)[:, np.newaxis]
            fmt = '.2f'
            title_suffix = ' (Normalized)'
        else:
            fmt = '.0f'
            title_suffix = ''
        
        # Plot heatmap
        sns.heatmap(cm_array, annot=True, fmt=fmt, cmap='Blues', 
                   xticklabels=class_names, yticklabels=class_names, ax=ax)
        ax.set_title(f'Fold {fold} Confusion Matrix{title_suffix}', fontweight='bold')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('Actual')
    
    # Add overall title
    fig.suptitle('Cross-Validation Results: Metrics and Confusion Matrices', 
                fontsize=16, fontweight='bold', y=0.98)
    
    # Add summary statistics as text
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    summary_text = f'Summary Statistics:\n'
    summary_text += f'Accuracy: {mean_acc:.3f} ± {std_acc:.3f}\n'
    summary_text += f'Macro F1: {mean_f1:.3f} ± {std_f1:.3f}'
    
    fig.text(0.98, 0.02, summary_text, fontsize=10, ha='right', va='bottom',
            bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Plot saved to: {save_path}")
    
    plt.show()


def compute_average_metrics(crossval_split_metrics: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    """
    Compute average metrics across all cross-validation folds.
    
    Parameters:
    -----------
    crossval_split_metrics : dict
        Dictionary containing metrics for each fold
        
    Returns:
    --------
    dict
        Dictionary with averaged metrics: {'avg_conf_matrix', 'avg_macro_f1', 'avg_acc'}
    """
    
    # Extract individual metrics
    accuracies = []
    f1_scores = []
    confusion_matrices = []
    
    for fold, metrics in crossval_split_metrics.items():
        # Extract accuracy
        acc = metrics['acc'][0] if isinstance(metrics['acc'], list) else metrics['acc']
        accuracies.append(acc)
        
        # Extract F1 score
        f1 = metrics['macro_f1'][0] if isinstance(metrics['macro_f1'], list) else metrics['macro_f1']
        f1_scores.append(f1)
        
        # Extract confusion matrix
        conf_matrix = metrics['conf_matrix']
        if hasattr(conf_matrix, 'values'):
            cm_array = conf_matrix.values
        else:
            cm_array = np.array(conf_matrix)
        confusion_matrices.append(cm_array)
    
    # Compute averages
    avg_accuracy = np.mean(accuracies)
    avg_f1 = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)
    
    # Create result dictionary
    avg_metrics = {
        'avg_acc': avg_accuracy,
        'avg_macro_f1': avg_f1,
        'avg_conf_matrix': avg_confusion_matrix
    }
    
    print(f"📊 Average Metrics Computed:")
    print(f"   Average Accuracy: {avg_accuracy:.4f}")
    print(f"   Average Macro F1: {avg_f1:.4f}")
    print(f"   Average Confusion Matrix shape: {avg_confusion_matrix.shape}")
    
    return avg_metrics


def plot_average_metrics(avg_metrics: Dict[str, Any], 
                        class_names: list = ['nf', 'hcm', 'dcm'],
                        normalize_cm: bool = False,
                        figsize: tuple = (12, 5),
                        save_path: Optional[str] = None) -> None:
    """
    Plot average metrics: bar chart for accuracy/F1 and heatmap for confusion matrix.
    
    Parameters:
    -----------
    avg_metrics : dict
        Dictionary containing averaged metrics
    class_names : list, default=['nf', 'hcm', 'dcm']
        Names of the classes for confusion matrix labels
    normalize_cm : bool, default=False
        Whether to normalize the confusion matrix
    figsize : tuple, default=(12, 5)
        Figure size for the plot
    save_path : str, optional
        Path to save the figure
    """
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Plot 1: Bar chart of average metrics
    metrics_names = ['Accuracy', 'Macro F1']
    metrics_values = [avg_metrics['avg_acc'], avg_metrics['avg_macro_f1']]
    colors = ['skyblue', 'lightcoral']
    
    bars = ax1.bar(metrics_names, metrics_values, color=colors, alpha=0.8, edgecolor='black')
    ax1.set_ylabel('Score')
    ax1.set_title('Average Cross-Validation Metrics', fontweight='bold')
    ax1.set_ylim(0, 1)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, metrics_values):
        height = bar.get_height()
        ax1.annotate(f'{value:.3f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom',
                    fontweight='bold')
    
    # Plot 2: Average confusion matrix
    cm_array = avg_metrics['avg_conf_matrix']
    
    # Normalize if requested
    if normalize_cm:
        cm_array = cm_array.astype('float') / cm_array.sum(axis=1)[:, np.newaxis]
        fmt = '.3f'
        title_suffix = ' (Normalized)'
        cmap = 'Blues'
    else:
        fmt = '.0f'
        title_suffix = ''
        cmap = 'Blues'
    
    # Plot heatmap
    sns.heatmap(cm_array, annot=True, fmt=fmt, cmap=cmap, 
               xticklabels=class_names, yticklabels=class_names, ax=ax2)
    ax2.set_title(f'Average Confusion Matrix{title_suffix}', fontweight='bold')
    ax2.set_xlabel('Predicted')
    ax2.set_ylabel('Actual')
    
    # Add summary statistics
    summary_text = f'Cross-Validation Summary:\n'
    summary_text += f'Accuracy: {avg_metrics["avg_acc"]:.4f}\n'
    summary_text += f'Macro F1: {avg_metrics["avg_macro_f1"]:.4f}'
    
    fig.text(0.02, 0.98, summary_text, fontsize=10, ha='left', va='top',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
            transform=fig.transFigure)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Average metrics plot saved to: {save_path}")
    
    plt.show()


def comprehensive_cv_analysis(crossval_split_metrics: Dict[str, Dict[str, Any]],
                             class_names: list = ['nf', 'hcm', 'dcm'],
                             normalize_cm: bool = False,
                             save_individual: Optional[str] = None,
                             save_average: Optional[str] = None) -> Dict[str, Any]:
    """
    Comprehensive analysis: plot individual fold results, compute averages, and plot averages.
    
    Parameters:
    -----------
    crossval_split_metrics : dict
        Dictionary containing metrics for each fold
    class_names : list, default=['nf', 'hcm', 'dcm']
        Names of the classes
    normalize_cm : bool, default=False
        Whether to normalize confusion matrices
    save_individual : str, optional
        Path to save individual folds plot
    save_average : str, optional
        Path to save average metrics plot
        
    Returns:
    --------
    dict
        Dictionary with averaged metrics
    """
    
    print("🔍 Starting comprehensive cross-validation analysis...")
    
    # Plot individual fold results
    print("\n📊 Plotting individual fold results...")
    plot_cv_metrics_and_confusion_matrices(
        crossval_split_metrics, 
        normalize_cm=normalize_cm, 
        save_path=save_individual
    )
    
    # Compute average metrics
    print("\n🧮 Computing average metrics...")
    avg_metrics = compute_average_metrics(crossval_split_metrics)
    
    # Plot average results
    print("\n📈 Plotting average results...")
    plot_average_metrics(
        avg_metrics, 
        class_names=class_names, 
        normalize_cm=normalize_cm, 
        save_path=save_average
    )
    
    # Print detailed summary
    print("\n" + "="*60)
    print("CROSS-VALIDATION SUMMARY")
    print("="*60)
    
    # Individual fold results
    folds = sorted([int(k) for k in crossval_split_metrics.keys()])
    print(f"\nIndividual Fold Results:")
    print(f"{'Fold':<6}{'Accuracy':<12}{'Macro F1':<12}")
    print("-" * 30)
    
    for fold in folds:
        fold_str = str(fold)
        acc = crossval_split_metrics[fold_str]['acc'][0] if isinstance(crossval_split_metrics[fold_str]['acc'], list) else crossval_split_metrics[fold_str]['acc']
        f1 = crossval_split_metrics[fold_str]['macro_f1'][0] if isinstance(crossval_split_metrics[fold_str]['macro_f1'], list) else crossval_split_metrics[fold_str]['macro_f1']
        print(f"{fold:<6}{acc:<12.4f}{f1:<12.4f}")
    
    # Summary statistics
    accs = [crossval_split_metrics[str(f)]['acc'][0] if isinstance(crossval_split_metrics[str(f)]['acc'], list) else crossval_split_metrics[str(f)]['acc'] for f in folds]
    f1s = [crossval_split_metrics[str(f)]['macro_f1'][0] if isinstance(crossval_split_metrics[str(f)]['macro_f1'], list) else crossval_split_metrics[str(f)]['macro_f1'] for f in folds]
    
    print(f"\nSummary Statistics:")
    print(f"Accuracy  - Mean: {np.mean(accs):.4f}, Std: {np.std(accs):.4f}, Min: {np.min(accs):.4f}, Max: {np.max(accs):.4f}")
    print(f"Macro F1  - Mean: {np.mean(f1s):.4f}, Std: {np.std(f1s):.4f}, Min: {np.min(f1s):.4f}, Max: {np.max(f1s):.4f}")
    
    print(f"\n✅ Analysis completed successfully!")
    
    return avg_metrics


# Example usage and testing
if __name__ == "__main__":
    
    
    # Run comprehensive analysis
    avg_results = comprehensive_cv_analysis(
        crossval_split_metrics,
        class_names=['nf', 'hcm', 'dcm'],
        normalize_cm=False,
        save_individual="cv_individual_results.png",
        save_average="cv_average_results.png"
    )
    
    print(f"\n🎯 Average metrics dictionary:")
    for key, value in avg_results.items():
        if key == 'avg_conf_matrix':
            print(f"{key}: shape {value.shape}")
        else:
            print(f"{key}: {value:.4f}")

In [ ]:
import pandas as pd
from datasets import Dataset
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def analyze_cell_types_by_individual(dataset_path):
    """
    Analyzes cell type diversity for each individual in the dataset.
    
    Args:
        dataset_path (str): Path to the HuggingFace dataset
    
    Returns:
        pd.DataFrame: Summary statistics of cell type diversity
    """
    
    print("Loading HuggingFace dataset...")
    try:
        # Load the dataset
        dataset = Dataset.load_from_disk(dataset_path)
        df = dataset.to_pandas()
        print(f"Dataset loaded successfully! Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        # Check if required columns exist
        required_cols = ['individual', 'cell_type']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if missing_cols:
            print(f"❌ Missing columns: {missing_cols}")
            print(f"Available columns: {df.columns.tolist()}")
            return None
        
        print(f"\n" + "="*80)
        print("CELL TYPE ANALYSIS BY INDIVIDUAL")
        print("="*80)
        
        # Get unique individuals
        unique_individuals = df['individual'].unique()
        print(f"Total unique individuals: {len(unique_individuals)}")
        
        # Analyze cell type diversity for each individual
        individual_analysis = []
        
        print(f"\nCell type diversity per individual:")
        print("-" * 60)
        
        for individual_id in sorted(unique_individuals):
            # Get all samples for this individual
            individual_data = df[df['individual'] == individual_id]
            
            # Get unique cell types for this individual
            unique_cell_types = individual_data['cell_type'].unique()
            num_unique_cell_types = len(unique_cell_types)
            total_samples = len(individual_data)
            
            # Get cell type distribution
            cell_type_counts = individual_data['cell_type'].value_counts()
            
            # Store summary info
            individual_analysis.append({
                'individual_id': individual_id,
                'total_samples': total_samples,
                'unique_cell_types': num_unique_cell_types,
                'cell_type_list': list(unique_cell_types),
                'most_common_cell_type': cell_type_counts.index[0],
                'most_common_count': cell_type_counts.iloc[0],
                'most_common_percentage': (cell_type_counts.iloc[0] / total_samples) * 100
            })
            
            print(f"Individual {individual_id}:")
            print(f"  Total samples: {total_samples:,}")
            print(f"  Unique cell types: {num_unique_cell_types}")
            print(f"  Cell types: {', '.join(sorted(unique_cell_types))}")
            print(f"  Most common: {cell_type_counts.index[0]} ({cell_type_counts.iloc[0]:,} samples, {(cell_type_counts.iloc[0]/total_samples)*100:.1f}%)")
            
            # Show detailed distribution if there are multiple cell types
            if num_unique_cell_types > 1:
                print(f"  Distribution:")
                for cell_type, count in cell_type_counts.items():
                    percentage = (count / total_samples) * 100
                    print(f"    {cell_type}: {count:,} ({percentage:.1f}%)")
            print()
        
        # Convert to DataFrame for easier analysis
        analysis_df = pd.DataFrame(individual_analysis)
        
        # Summary statistics
        print(f"\n" + "="*80)
        print("SUMMARY STATISTICS")
        print("="*80)
        
        print(f"\nCell type diversity statistics:")
        print(f"  Min cell types per individual: {analysis_df['unique_cell_types'].min()}")
        print(f"  Max cell types per individual: {analysis_df['unique_cell_types'].max()}")
        print(f"  Mean cell types per individual: {analysis_df['unique_cell_types'].mean():.2f}")
        print(f"  Median cell types per individual: {analysis_df['unique_cell_types'].median():.1f}")
        
        # Distribution of cell type counts
        cell_type_dist = analysis_df['unique_cell_types'].value_counts().sort_index()
        print(f"\nDistribution of cell type diversity:")
        for num_types, count in cell_type_dist.items():
            percentage = (count / len(analysis_df)) * 100
            print(f"  {num_types} cell types: {count} individuals ({percentage:.1f}%)")
        
        # Most common cell types across all individuals
        print(f"\nMost common cell types across all individuals:")
        most_common_overall = analysis_df['most_common_cell_type'].value_counts()
        for cell_type, count in most_common_overall.items():
            percentage = (count / len(analysis_df)) * 100
            print(f"  {cell_type}: dominant in {count} individuals ({percentage:.1f}%)")
        
        # Get all unique cell types in the dataset
        all_cell_types = df['cell_type'].unique()
        print(f"\nOverall dataset cell type information:")
        print(f"  Total unique cell types in dataset: {len(all_cell_types)}")
        print(f"  All cell types: {', '.join(sorted(all_cell_types))}")
        
        # Cell type frequency across the entire dataset
        print(f"\nCell type frequency across entire dataset:")
        overall_cell_type_counts = df['cell_type'].value_counts()
        total_samples_dataset = len(df)
        for cell_type, count in overall_cell_type_counts.items():
            percentage = (count / total_samples_dataset) * 100
            print(f"  {cell_type}: {count:,} samples ({percentage:.1f}%)")
        
        # Create visualizations
        print(f"\n" + "="*80)
        print("CREATING VISUALIZATIONS")
        print("="*80)
        
        # Set up plotting
        plt.style.use('default')
        sns.set_palette("husl")
        
        # Create figure with subplots
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Cell Type Analysis by Individual', fontsize=16, fontweight='bold')
        
        # Plot 1: Distribution of cell type diversity
        ax1 = axes[0, 0]
        cell_type_dist.plot(kind='bar', ax=ax1, color='skyblue', alpha=0.7)
        ax1.set_title('Distribution of Cell Type Diversity per Individual', fontweight='bold')
        ax1.set_xlabel('Number of Unique Cell Types')
        ax1.set_ylabel('Number of Individuals')
        ax1.grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for i, v in enumerate(cell_type_dist.values):
            ax1.text(i, v + 0.1, str(v), ha='center', va='bottom', fontweight='bold')
        
        # Plot 2: Sample count vs cell type diversity
        ax2 = axes[0, 1]
        scatter = ax2.scatter(analysis_df['total_samples'], analysis_df['unique_cell_types'], 
                             alpha=0.6, s=60, c=analysis_df['unique_cell_types'], 
                             cmap='viridis')
        ax2.set_title('Sample Count vs Cell Type Diversity', fontweight='bold')
        ax2.set_xlabel('Total Samples per Individual')
        ax2.set_ylabel('Number of Unique Cell Types')
        ax2.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax2, label='Cell Type Count')
        
        # Plot 3: Most common cell types across individuals
        ax3 = axes[1, 0]
        most_common_overall.plot(kind='bar', ax=ax3, color='lightcoral', alpha=0.7)
        ax3.set_title('Most Dominant Cell Type per Individual', fontweight='bold')
        ax3.set_xlabel('Cell Type')
        ax3.set_ylabel('Number of Individuals')
        ax3.tick_params(axis='x', rotation=45)
        ax3.grid(axis='y', alpha=0.3)
        
        # Add value labels
        for i, v in enumerate(most_common_overall.values):
            ax3.text(i, v + 0.1, str(v), ha='center', va='bottom', fontweight='bold')
        
        # Plot 4: Overall cell type distribution in dataset
        ax4 = axes[1, 1]
        # For better visualization, show top 10 cell types
        top_cell_types = overall_cell_type_counts.head(10)
        top_cell_types.plot(kind='bar', ax=ax4, color='lightgreen', alpha=0.7)
        ax4.set_title('Top 10 Cell Types in Entire Dataset', fontweight='bold')
        ax4.set_xlabel('Cell Type')
        ax4.set_ylabel('Number of Samples')
        ax4.tick_params(axis='x', rotation=45)
        ax4.grid(axis='y', alpha=0.3)
        
        # Add percentage labels
        for i, (cell_type, count) in enumerate(top_cell_types.items()):
            pct = (count / total_samples_dataset) * 100
            ax4.text(i, count + count*0.01, f'{pct:.1f}%', 
                    ha='center', va='bottom', fontweight='bold', fontsize=8)
        
        plt.tight_layout()
        
        # Save the plot
        plot_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/cell_type_analysis.png"
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"Visualizations saved to: {plot_path}")
        
        plt.show()
        
        # Save detailed results
        csv_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/cell_type_analysis_detailed.csv"
        analysis_df.to_csv(csv_path, index=False)
        print(f"Detailed analysis saved to: {csv_path}")
        
        print(f"\n✅ Cell type analysis completed successfully!")
        return analysis_df
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Main execution
if __name__ == "__main__":
    dataset_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/human_dcm_hcm_nf.dataset/"
    
    print("Starting cell type analysis...")
    results = analyze_cell_types_by_individual(dataset_path)
    
    if results is not None:
        print(f"\nAnalysis completed! Results shape: {results.shape}")
        print("\nSample of results:")
        print(results.head())

In [ ]:
import os
from datasets import Dataset
import pandas as pd
from collections import Counter

# Read the HuggingFace dataset
dataset_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/human_dcm_hcm_nf.dataset/"

print("Loading dataset...")
try:
    # Load the dataset
    dataset = Dataset.load_from_disk(dataset_path)
    print(f"Dataset loaded successfully!")
    print(f"Dataset length: {len(dataset)}")
    print(f"Dataset features: {list(dataset.features.keys())}")
    print()
    
    # Convert to pandas for easier analysis
    df = dataset.to_pandas()
    
    # Check if required columns exist
    if 'individual' not in df.columns or 'disease' not in df.columns:
        print("Available columns:", df.columns.tolist())
        print("Please check if 'individual' and 'disease' columns exist with these exact names")
    else:
        print("=== ANALYSIS RESULTS ===")
        print()
        
        # 1. Number of unique individuals
        unique_individuals = df['individual'].nunique()
        print(f"Number of unique individuals: {unique_individuals}")
        print()
        
        # 2. List all unique individuals
        individual_ids = df['individual'].unique()
        print("Unique individual IDs:")
        for i, ind_id in enumerate(sorted(individual_ids), 1):
            print(f"  {i}. {ind_id}")
        print()
        
        # 3. Overall disease distribution
        disease_counts = df['disease'].value_counts()
        print("Overall disease distribution:")
        for disease, count in disease_counts.items():
            print(f"  {disease}: {count} samples")
        print()
        
        # 4. Disease distribution per individual
        print("Disease distribution per individual:")
        print("=" * 50)
        
        for ind_id in sorted(individual_ids):
            individual_data = df[df['individual'] == ind_id]
            disease_dist = individual_data['disease'].value_counts()
            total_samples = len(individual_data)
            
            print(f"\nIndividual ID: {ind_id}")
            print(f"Total samples: {total_samples}")
            print("Disease distribution:")
            for disease, count in disease_dist.items():
                percentage = (count / total_samples) * 100
                print(f"  - {disease}: {count} samples ({percentage:.1f}%)")
        
        # 5. Summary statistics
        print("\n" + "=" * 50)
        print("SUMMARY STATISTICS")
        print("=" * 50)
        
        # Samples per individual
        samples_per_individual = df.groupby('individual').size()
        print(f"\nSamples per individual:")
        print(f"  Min: {samples_per_individual.min()}")
        print(f"  Max: {samples_per_individual.max()}")
        print(f"  Mean: {samples_per_individual.mean():.1f}")
        print(f"  Median: {samples_per_individual.median():.1f}")
        
        # Cross-tabulation
        print(f"\nCross-tabulation (Individual vs Disease):")
        crosstab = pd.crosstab(df['individual'], df['disease'], margins=True)
        print(crosstab)
        
        # 6. Check for any individuals with multiple diseases
        print(f"\nIndividuals with multiple disease types:")
        multi_disease_individuals = df.groupby('individual')['disease'].nunique()
        multi_disease = multi_disease_individuals[multi_disease_individuals > 1]
        
        if len(multi_disease) > 0:
            for ind_id, num_diseases in multi_disease.items():
                diseases = df[df['individual'] == ind_id]['disease'].unique()
                print(f"  {ind_id}: {num_diseases} diseases - {list(diseases)}")
        else:
            print("  No individuals have multiple disease types")

except Exception as e:
    print(f"Error loading dataset: {str(e)}")
    print()
    print("Troubleshooting steps:")
    print("1. Check if the path exists")
    print("2. Verify the dataset format")
    print("3. Check column names")
    
    # Try to list directory contents
    try:
        if os.path.exists(dataset_path):
            contents = os.listdir(dataset_path)
            print(f"Directory contents: {contents}")
        else:
            print("Dataset path does not exist")
    except Exception as e2:
        print(f"Cannot access directory: {str(e2)}")

In [ ]:
import json
import numpy as np
from collections import defaultdict
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Individual IDs and their diseases from your data
individuals_diseases = {
    "1290": 'dcm', "1300": 'dcm', "1304": 'dcm', "1358": 'dcm', "1371": 'dcm',
    "1430": 'dcm', "1437": 'dcm', "1472": 'dcm', "1504": 'dcm', "1606": 'dcm',
    "1617": 'dcm', "1422": 'hcm', "1425": 'hcm', "1447": 'hcm', "1462": 'hcm',
    '1479': 'hcm', "1508": 'hcm', "1510": 'hcm', "1602": 'hcm', "1630": 'hcm',
    "1631": 'hcm', "1685": 'hcm', "1707": 'hcm', "1722": 'hcm', "1726": 'hcm',
    "1735": 'hcm', "1515": 'nf', "1516": 'nf', "1539": 'nf', "1540": 'nf',
    "1547": 'nf', "1549": 'nf', "1558": 'nf', "1561": 'nf', "1582": 'nf',
    "1600": 'nf', "1603": 'nf', "1610": 'nf', "1622": 'nf', "1678": 'nf',
    "1702": 'nf', "1718": 'nf'
}

# Group individuals by disease
disease_groups = defaultdict(list)
for individual_id, disease in individuals_diseases.items():
    disease_groups[disease].append(individual_id)

print("Disease distribution:")
for disease, individuals in disease_groups.items():
    print(f"{disease}: {len(individuals)} individuals")

# Create 5 folds with balanced disease distribution
def create_balanced_folds(disease_groups, n_folds=5):
    folds = [[] for _ in range(n_folds)]
    
    # For each disease, distribute individuals across folds
    for disease, individuals in disease_groups.items():
        # Shuffle individuals within each disease group
        shuffled_individuals = individuals.copy()
        random.shuffle(shuffled_individuals)
        
        # Distribute individuals across folds in round-robin fashion
        for i, individual in enumerate(shuffled_individuals):
            fold_idx = i % n_folds
            folds[fold_idx].append(individual)
    
    return folds

# Create the folds
folds = create_balanced_folds(disease_groups)

# Print fold statistics
print("\nFold statistics:")
for i, fold in enumerate(folds):
    fold_diseases = [individuals_diseases[ind] for ind in fold]
    disease_counts = {disease: fold_diseases.count(disease) for disease in ['dcm', 'hcm', 'nf']}
    print(f"Fold {i+1}: {len(fold)} individuals - DCM: {disease_counts['dcm']}, HCM: {disease_counts['hcm']}, NF: {disease_counts['nf']}")

# Create the cross-validation splits in the required format
cv_splits = {}

for fold_idx in range(5):
    # Current fold becomes validation set
    eval_individuals = folds[fold_idx]
    
    # All other folds become training set
    train_individuals = []
    for other_fold_idx in range(5):
        if other_fold_idx != fold_idx:
            train_individuals.extend(folds[other_fold_idx])
    
    cv_splits[str(fold_idx + 1)] = {
        "attr_key": "individual",
        "train": train_individuals,
        "eval": eval_individuals
    }

# Save to JSON file
output_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/5fold_cv_splits.json"

with open(output_path, 'w') as f:
    json.dump(cv_splits, f, indent=2)

print(f"\n5-fold cross-validation splits saved to: {output_path}")

# Print summary of each fold
print("\nDetailed fold information:")
for fold_num, fold_data in cv_splits.items():
    print(f"\nFold {fold_num}:")
    print(f"  Training set: {len(fold_data['train'])} individuals")
    print(f"  Validation set: {len(fold_data['eval'])} individuals")
    
    # Count diseases in training set
    train_diseases = [individuals_diseases[ind] for ind in fold_data['train']]
    train_disease_counts = {disease: train_diseases.count(disease) for disease in ['dcm', 'hcm', 'nf']}
    print(f"  Training diseases - DCM: {train_disease_counts['dcm']}, HCM: {train_disease_counts['hcm']}, NF: {train_disease_counts['nf']}")
    
    # Count diseases in validation set
    eval_diseases = [individuals_diseases[ind] for ind in fold_data['eval']]
    eval_disease_counts = {disease: eval_diseases.count(disease) for disease in ['dcm', 'hcm', 'nf']}
    print(f"  Validation diseases - DCM: {eval_disease_counts['dcm']}, HCM: {eval_disease_counts['hcm']}, NF: {eval_disease_counts['nf']}")

print("\nCross-validation splits created successfully!")

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from collections import Counter
import numpy as np

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# Paths
dataset_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/human_dcm_hcm_nf.dataset/"
json_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/5fold_cv_splits.json"

print("Loading HuggingFace dataset...")
try:
    # Load the dataset
    dataset = Dataset.load_from_disk(dataset_path)
    df = dataset.to_pandas()
    print(f"Dataset loaded successfully! Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Load JSON splits
    print("\nLoading cross-validation splits...")
    with open(json_path, 'r') as f:
        cv_splits = json.load(f)
    
    print(f"Loaded {len(cv_splits)} folds")
    
    # Check for data leakage across all folds
    print("\n" + "="*60)
    print("CHECKING FOR DATA LEAKAGE")
    print("="*60)
    
    all_train_ids = set()
    all_eval_ids = set()
    leakage_found = False
    
    for fold_num, fold_data in cv_splits.items():
        train_ids = set(fold_data['train'])
        eval_ids = set(fold_data['eval'])
        
        # Check for overlap within the same fold
        overlap = train_ids.intersection(eval_ids)
        if overlap:
            print(f"❌ LEAKAGE DETECTED in Fold {fold_num}: {len(overlap)} individuals in both train and eval")
            print(f"   Overlapping IDs: {list(overlap)}")
            leakage_found = True
        else:
            print(f"✅ Fold {fold_num}: No overlap between train and eval sets")
        
        all_train_ids.update(train_ids)
        all_eval_ids.update(eval_ids)
    
    # Check if any individual appears in multiple eval sets
    eval_counts = Counter()
    for fold_num, fold_data in cv_splits.items():
        for individual in fold_data['eval']:
            eval_counts[individual] += 1
    
    multi_eval = {ind: count for ind, count in eval_counts.items() if count > 1}
    if multi_eval:
        print(f"❌ LEAKAGE DETECTED: {len(multi_eval)} individuals appear in multiple eval sets")
        for ind, count in multi_eval.items():
            print(f"   Individual {ind} appears in {count} eval sets")
        leakage_found = True
    
    if not leakage_found:
        print("✅ NO DATA LEAKAGE DETECTED: All splits are properly separated")
    
    print(f"\nTotal unique individuals in training across all folds: {len(all_train_ids)}")
    print(f"Total unique individuals in evaluation across all folds: {len(all_eval_ids)}")
    
    # Create visualization
    print("\n" + "="*60)
    print("CREATING VISUALIZATION")
    print("="*60)
    
    # Create subplots: 2 rows, 3 columns (5 folds + 1 overall)
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Class Distribution Across 5-Fold Cross-Validation Splits', fontsize=16, fontweight='bold')
    
    # Flatten axes for easier indexing
    axes = axes.flatten()
    
    # Define colors for consistency
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']  # Red, Teal, Blue
    disease_order = ['dcm', 'hcm', 'nf']
    
    # Plot each fold
    for i, (fold_num, fold_data) in enumerate(cv_splits.items()):
        ax = axes[i]
        
        # Get data for this fold
        train_df = df[df['individual'].isin(fold_data['train'])]
        eval_df = df[df['individual'].isin(fold_data['eval'])]
        
        # Count samples by disease
        train_counts = train_df['disease'].value_counts()
        eval_counts = eval_df['disease'].value_counts()
        
        # Ensure all diseases are represented (even if count is 0)
        train_counts = train_counts.reindex(disease_order, fill_value=0)
        eval_counts = eval_counts.reindex(disease_order, fill_value=0)
        
        # Create bar plot
        x = np.arange(len(disease_order))
        width = 0.35
        
        bars1 = ax.bar(x - width/2, train_counts.values, width, label='Train', 
                      color=colors, alpha=0.8)
        bars2 = ax.bar(x + width/2, eval_counts.values, width, label='Eval', 
                      color=colors, alpha=0.5)
        
        # Add value labels on bars
        for bar in bars1:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{int(height)}', ha='center', va='bottom', fontweight='bold')
        
        for bar in bars2:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{int(height)}', ha='center', va='bottom', fontweight='bold')
        
        # Customize subplot
        ax.set_title(f'Fold {fold_num}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Disease Type', fontsize=10)
        ax.set_ylabel('Number of Samples', fontsize=10)
        ax.set_xticks(x)
        ax.set_xticklabels(disease_order, fontsize=10)
        ax.legend(fontsize=9)
        ax.grid(axis='y', alpha=0.3)
        
        # Add fold statistics as text
        train_total = len(train_df)
        eval_total = len(eval_df)
        train_individuals = len(fold_data['train'])
        eval_individuals = len(fold_data['eval'])
        
        stats_text = f'Train: {train_total:,} samples ({train_individuals} individuals)\n'
        stats_text += f'Eval: {eval_total:,} samples ({eval_individuals} individuals)'
        
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                fontsize=8)
    
    # Create overall statistics plot in the last subplot
    ax_overall = axes[5]
    
    # Calculate overall statistics
    overall_train_counts = df['disease'].value_counts().reindex(disease_order, fill_value=0)
    
    # Calculate average eval counts across folds
    avg_eval_counts = []
    for disease in disease_order:
        total_eval = sum(df[df['individual'].isin(fold_data['eval'])]['disease'].value_counts().get(disease, 0) 
                        for fold_data in cv_splits.values())
        avg_eval_counts.append(total_eval / len(cv_splits))
    
    # Plot overall statistics
    x = np.arange(len(disease_order))
    bars1 = ax_overall.bar(x - width/2, overall_train_counts.values, width, 
                          label='Total Dataset', color=colors, alpha=0.8)
    bars2 = ax_overall.bar(x + width/2, avg_eval_counts, width, 
                          label='Avg Eval per Fold', color=colors, alpha=0.5)
    
    # Add value labels
    for bar in bars1:
        height = bar.get_height()
        ax_overall.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height)}', ha='center', va='bottom', fontweight='bold')
    
    for bar in bars2:
        height = bar.get_height()
        ax_overall.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height)}', ha='center', va='bottom', fontweight='bold')
    
    ax_overall.set_title('Overall Dataset Statistics', fontsize=12, fontweight='bold')
    ax_overall.set_xlabel('Disease Type', fontsize=10)
    ax_overall.set_ylabel('Number of Samples', fontsize=10)
    ax_overall.set_xticks(x)
    ax_overall.set_xticklabels(disease_order, fontsize=10)
    ax_overall.legend(fontsize=9)
    ax_overall.grid(axis='y', alpha=0.3)
    
    # Add overall statistics text
    total_samples = len(df)
    total_individuals = df['individual'].nunique()
    stats_text = f'Total: {total_samples:,} samples\n'
    stats_text += f'Individuals: {total_individuals}\n'
    stats_text += f'Avg per fold: {total_samples//5:,} samples'
    
    ax_overall.text(0.02, 0.98, stats_text, transform=ax_overall.transAxes, 
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                   fontsize=8)
    
    plt.tight_layout()
    
    # Save the plot
    plot_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/genecorpus_heart_disease/cv_splits_visualization.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"Visualization saved to: {plot_path}")
    
    plt.show()
    
    # Print detailed statistics
    print("\n" + "="*60)
    print("DETAILED FOLD STATISTICS")
    print("="*60)
    
    for fold_num, fold_data in cv_splits.items():
        print(f"\nFold {fold_num}:")
        
        train_df = df[df['individual'].isin(fold_data['train'])]
        eval_df = df[df['individual'].isin(fold_data['eval'])]
        
        print(f"  Training:")
        print(f"    Individuals: {len(fold_data['train'])}")
        print(f"    Samples: {len(train_df):,}")
        train_dist = train_df['disease'].value_counts()
        for disease in disease_order:
            count = train_dist.get(disease, 0)
            pct = (count / len(train_df)) * 100 if len(train_df) > 0 else 0
            print(f"    {disease.upper()}: {count:,} ({pct:.1f}%)")
        
        print(f"  Evaluation:")
        print(f"    Individuals: {len(fold_data['eval'])}")
        print(f"    Samples: {len(eval_df):,}")
        eval_dist = eval_df['disease'].value_counts()
        for disease in disease_order:
            count = eval_dist.get(disease, 0)
            pct = (count / len(eval_df)) * 100 if len(eval_df) > 0 else 0
            print(f"    {disease.upper()}: {count:,} ({pct:.1f}%)")
    
    print(f"\n✅ Analysis completed successfully!")
    print(f"📊 Visualization saved to: {plot_path}")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

# COVID Dataset from CellNexus

In [ ]:
import os
from datasets import Dataset
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Read the HuggingFace dataset
dataset_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/tokenized_30M/cellnexus_singlecell.dataset/"

print("Loading CellNexus COVID dataset...")
try:
    # Load the dataset
    dataset = Dataset.load_from_disk(dataset_path)
    print(f"Dataset loaded successfully!")
    print(f"Dataset length: {len(dataset):,}")
    print(f"Dataset features: {list(dataset.features.keys())}")
    print()
    
    # Convert to pandas for easier analysis
    df = dataset.to_pandas()
    print(f"DataFrame shape: {df.shape}")
    print()
    
    # Check if required columns exist
    if 'dataset_id' not in df.columns or 'disease' not in df.columns:
        print("Available columns:", df.columns.tolist())
        print("Please check if 'dataset_id' and 'disease' columns exist with these exact names")
        
        # Show first few rows to understand the structure
        print("\nFirst 5 rows of the dataset:")
        print(df.head())
        
    else:
        print("=== ANALYSIS RESULTS ===")
        print()
        
        # 1. Number of unique dataset_ids
        unique_datasets = df['dataset_id'].nunique()
        print(f"Number of unique dataset IDs: {unique_datasets}")
        print()
        
        # 2. List all unique dataset_ids
        dataset_ids = df['dataset_id'].unique()
        print("Unique dataset IDs:")
        for i, dataset_id in enumerate(sorted(dataset_ids), 1):
            print(f"  {i:2d}. {dataset_id}")
        print()
        
        # 3. Overall disease distribution
        disease_counts = df['disease'].value_counts()
        print("Overall disease distribution:")
        total_samples = len(df)
        for disease, count in disease_counts.items():
            percentage = (count / total_samples) * 100
            print(f"  {disease}: {count:,} samples ({percentage:.1f}%)")
        print()
        
        # 4. Disease distribution per dataset_id
        print("Disease distribution per dataset ID:")
        print("=" * 60)
        
        dataset_stats = []
        for dataset_id in sorted(dataset_ids):
            dataset_data = df[df['dataset_id'] == dataset_id]
            disease_dist = dataset_data['disease'].value_counts()
            total_samples = len(dataset_data)
            
            print(f"\nDataset ID: {dataset_id}")
            print(f"Total samples: {total_samples:,}")
            print("Disease distribution:")
            
            dataset_stat = {'dataset_id': dataset_id, 'total_samples': total_samples}
            for disease, count in disease_dist.items():
                percentage = (count / total_samples) * 100
                print(f"  - {disease}: {count:,} samples ({percentage:.1f}%)")
                dataset_stat[disease] = count
            
            # Fill missing diseases with 0
            for disease in disease_counts.index:
                if disease not in dataset_stat:
                    dataset_stat[disease] = 0
            
            dataset_stats.append(dataset_stat)
        
        # 5. Summary statistics
        print("\n" + "=" * 60)
        print("SUMMARY STATISTICS")
        print("=" * 60)
        
        # Samples per dataset_id
        samples_per_dataset = df.groupby('dataset_id').size()
        print(f"\nSamples per dataset:")
        print(f"  Min: {samples_per_dataset.min():,}")
        print(f"  Max: {samples_per_dataset.max():,}")
        print(f"  Mean: {samples_per_dataset.mean():.1f}")
        print(f"  Median: {samples_per_dataset.median():.1f}")
        print(f"  Std: {samples_per_dataset.std():.1f}")
        
        # Show top 10 largest and smallest datasets
        print(f"\nTop 10 largest datasets:")
        largest_datasets = samples_per_dataset.nlargest(10)
        for dataset_id, count in largest_datasets.items():
            print(f"  {dataset_id}: {count:,} samples")
        
        print(f"\nTop 10 smallest datasets:")
        smallest_datasets = samples_per_dataset.nsmallest(10)
        for dataset_id, count in smallest_datasets.items():
            print(f"  {dataset_id}: {count:,} samples")
        
        # Cross-tabulation
        print(f"\nCross-tabulation (Dataset ID vs Disease):")
        crosstab = pd.crosstab(df['dataset_id'], df['disease'], margins=True)
        print("Cross-tabulation summary (showing marginals only due to size):")
        print(f"Total datasets: {len(crosstab) - 1}")  # -1 for the 'All' row
        print(f"Disease totals:")
        for disease in disease_counts.index:
            print(f"  {disease}: {crosstab.loc['All', disease]:,} samples")
        
        # 6. Check for any datasets with multiple diseases
        print(f"\nDatasets with multiple disease types:")
        multi_disease_datasets = df.groupby('dataset_id')['disease'].nunique()
        multi_disease = multi_disease_datasets[multi_disease_datasets > 1]
        
        if len(multi_disease) > 0:
            print(f"Found {len(multi_disease)} datasets with multiple disease types:")
            for dataset_id, num_diseases in multi_disease.items():
                diseases = df[df['dataset_id'] == dataset_id]['disease'].unique()
                sample_count = len(df[df['dataset_id'] == dataset_id])
                print(f"  {dataset_id}: {num_diseases} diseases - {list(diseases)} ({sample_count:,} samples)")
        else:
            print("  No datasets have multiple disease types")
        
        # 7. Disease distribution across datasets
        print(f"\nDisease distribution across datasets:")
        disease_dataset_counts = df.groupby('disease')['dataset_id'].nunique()
        for disease, dataset_count in disease_dataset_counts.items():
            print(f"  {disease}: appears in {dataset_count} datasets")
        
        # 8. Create visualizations
        print(f"\n" + "=" * 60)
        print("CREATING VISUALIZATIONS")
        print("=" * 60)
        
        # Set up the plotting style
        plt.style.use('default')
        sns.set_palette("husl")
        
        # Create a figure with multiple subplots
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('CellNexus COVID Dataset Analysis', fontsize=16, fontweight='bold')
        
        # 1. Overall disease distribution (pie chart)
        ax1 = axes[0, 0]
        disease_counts.plot(kind='pie', ax=ax1, autopct='%1.1f%%', startangle=90)
        ax1.set_title('Overall Disease Distribution', fontweight='bold')
        ax1.set_ylabel('')
        
        # 2. Samples per dataset distribution (histogram)
        ax2 = axes[0, 1]
        ax2.hist(samples_per_dataset.values, bins=50, alpha=0.7, edgecolor='black')
        ax2.set_title('Distribution of Sample Counts per Dataset', fontweight='bold')
        ax2.set_xlabel('Number of Samples')
        ax2.set_ylabel('Number of Datasets')
        ax2.set_yscale('log')  # Log scale due to potentially wide range
        
        # 3. Disease distribution by dataset (stacked bar for top datasets)
        ax3 = axes[1, 0]
        top_datasets = samples_per_dataset.nlargest(20)
        top_dataset_data = []
        for dataset_id in top_datasets.index:
            dataset_diseases = df[df['dataset_id'] == dataset_id]['disease'].value_counts()
            row_data = {'dataset_id': dataset_id}
            for disease in disease_counts.index:
                row_data[disease] = dataset_diseases.get(disease, 0)
            top_dataset_data.append(row_data)
        
        top_df = pd.DataFrame(top_dataset_data)
        top_df.set_index('dataset_id', inplace=True)
        
        top_df.plot(kind='bar', stacked=True, ax=ax3, width=0.8)
        ax3.set_title('Disease Distribution in Top 20 Largest Datasets', fontweight='bold')
        ax3.set_xlabel('Dataset ID')
        ax3.set_ylabel('Number of Samples')
        ax3.tick_params(axis='x', rotation=45)
        ax3.legend(title='Disease', bbox_to_anchor=(1.05, 1), loc='upper left')
        
        # 4. Box plot of sample sizes by disease
        ax4 = axes[1, 1]
        dataset_disease_sizes = []
        for dataset_id in dataset_ids:
            dataset_data = df[df['dataset_id'] == dataset_id]
            disease = dataset_data['disease'].iloc[0]  # Since each dataset has only one disease
            size = len(dataset_data)
            dataset_disease_sizes.append({'disease': disease, 'size': size})
        
        size_df = pd.DataFrame(dataset_disease_sizes)
        sns.boxplot(data=size_df, x='disease', y='size', ax=ax4)
        ax4.set_title('Dataset Size Distribution by Disease', fontweight='bold')
        ax4.set_xlabel('Disease')
        ax4.set_ylabel('Number of Samples (log scale)')
        ax4.set_yscale('log')
        
        plt.tight_layout()
        
        # Save the plot
        plot_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/dataset_analysis.png"
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"Visualizations saved to: {plot_path}")
        
        plt.show()
        
        # 9. Export summary statistics
        print(f"\n" + "=" * 60)
        print("EXPORTING SUMMARY")
        print("=" * 60)
        
        # Create summary DataFrame
        summary_df = pd.DataFrame(dataset_stats)
        summary_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/dataset_summary.csv"
        summary_df.to_csv(summary_path, index=False)
        print(f"Dataset summary exported to: {summary_path}")
        
        # Print final summary
        print(f"\n📊 FINAL SUMMARY:")
        print(f"  • Total samples: {len(df):,}")
        print(f"  • Total datasets: {unique_datasets}")
        print(f"  • Disease types: {len(disease_counts)}")
        print(f"  • Diseases: {', '.join(disease_counts.index)}")
        print(f"  • Average samples per dataset: {samples_per_dataset.mean():.1f}")
        print(f"  • Datasets with mixed diseases: {len(multi_disease)}")

except Exception as e:
    print(f"❌ Error loading dataset: {str(e)}")
    print()
    print("Troubleshooting steps:")
    print("1. Check if the path exists")
    print("2. Verify the dataset format")
    print("3. Check column names")
    
    # Try to list directory contents
    try:
        if os.path.exists(dataset_path):
            contents = os.listdir(dataset_path)
            print(f"Directory contents: {contents}")
        else:
            print("Dataset path does not exist")
    except Exception as e2:
        print(f"Cannot access directory: {str(e2)}")
    
    import traceback
    traceback.print_exc()

In [ ]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Read the dataset summary CSV
csv_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/dataset_summary.csv"

print("Loading CellNexus COVID dataset summary...")
try:
    # Load the CSV data
    df = pd.read_csv(csv_path)
    print(f"Dataset summary loaded successfully! Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Display basic info
    print(f"\nDataset overview:")
    print(f"Total datasets: {len(df)}")
    print(f"Total samples: {df['total_samples'].sum():,}")
    print(f"Total COVID-19 samples: {df['COVID-19'].sum():,}")
    print(f"Total normal samples: {df['normal'].sum():,}")
    
    # Categorize datasets by disease composition
    covid_only = df[(df['COVID-19'] > 0) & (df['normal'] == 0)]
    normal_only = df[(df['normal'] > 0) & (df['COVID-19'] == 0)]
    mixed = df[(df['COVID-19'] > 0) & (df['normal'] > 0)]
    
    print(f"\nDataset categorization:")
    print(f"COVID-19 only: {len(covid_only)} datasets")
    print(f"Normal only: {len(normal_only)} datasets")
    print(f"Mixed: {len(mixed)} datasets")
    
    # For stratified splits, we'll consider the dominant disease in each dataset
    # For mixed datasets, we'll classify based on the majority class
    dataset_disease_mapping = {}
    
    for _, row in df.iterrows():
        dataset_id = row['dataset_id']
        covid_count = row['COVID-19']
        normal_count = row['normal']
        
        if covid_count > normal_count:
            dataset_disease_mapping[dataset_id] = 'COVID-19'
        elif normal_count > covid_count:
            dataset_disease_mapping[dataset_id] = 'normal'
        else:
            # Equal counts (rare case) - assign to COVID-19
            dataset_disease_mapping[dataset_id] = 'COVID-19'
    
    # Group datasets by their dominant disease
    disease_groups = defaultdict(list)
    for dataset_id, disease in dataset_disease_mapping.items():
        disease_groups[disease].append(dataset_id)
    
    print(f"\nDataset grouping by dominant disease:")
    for disease, datasets in disease_groups.items():
        print(f"{disease}: {len(datasets)} datasets")
        for dataset_id in datasets:
            row = df[df['dataset_id'] == dataset_id].iloc[0]
            print(f"  {dataset_id}: {row['COVID-19']} COVID + {row['normal']} normal = {row['total_samples']} total")
    
    # Create 5 folds with balanced disease distribution
    def create_balanced_folds(disease_groups, n_folds=5):
        folds = [[] for _ in range(n_folds)]
        
        # For each disease, distribute datasets across folds
        for disease, datasets in disease_groups.items():
            # Shuffle datasets within each disease group
            shuffled_datasets = datasets.copy()
            random.shuffle(shuffled_datasets)
            
            # Distribute datasets across folds in round-robin fashion
            for i, dataset_id in enumerate(shuffled_datasets):
                fold_idx = i % n_folds
                folds[fold_idx].append(dataset_id)
        
        return folds
    
    # Create the folds
    folds = create_balanced_folds(disease_groups)
    
    # Print fold statistics
    print(f"\nFold statistics:")
    for i, fold in enumerate(folds):
        fold_diseases = [dataset_disease_mapping[dataset_id] for dataset_id in fold]
        disease_counts = {disease: fold_diseases.count(disease) for disease in ['COVID-19', 'normal']}
        
        # Calculate total samples in this fold
        fold_samples = df[df['dataset_id'].isin(fold)]['total_samples'].sum()
        fold_covid = df[df['dataset_id'].isin(fold)]['COVID-19'].sum()
        fold_normal = df[df['dataset_id'].isin(fold)]['normal'].sum()
        
        print(f"Fold {i+1}: {len(fold)} datasets")
        print(f"  Dominant diseases - COVID-19: {disease_counts['COVID-19']}, Normal: {disease_counts['normal']}")
        print(f"  Sample counts - COVID-19: {fold_covid:,}, Normal: {fold_normal:,}, Total: {fold_samples:,}")
    
    # Create the cross-validation splits in the required format
    cv_splits = {}
    
    for fold_idx in range(5):
        # Current fold becomes validation set
        eval_datasets = folds[fold_idx]
        
        # All other folds become training set
        train_datasets = []
        for other_fold_idx in range(5):
            if other_fold_idx != fold_idx:
                train_datasets.extend(folds[other_fold_idx])
        
        cv_splits[str(fold_idx + 1)] = {
            "attr_key": "dataset_id",
            "train": train_datasets,
            "eval": eval_datasets
        }
    
    # Save to JSON file
    output_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/5fold_cv_splits.json"
    
    with open(output_path, 'w') as f:
        json.dump(cv_splits, f, indent=2)
    
    print(f"\n5-fold cross-validation splits saved to: {output_path}")
    
    # Detailed analysis of each fold
    print(f"\n" + "=" * 80)
    print("DETAILED FOLD ANALYSIS")
    print("=" * 80)
    
    for fold_num, fold_data in cv_splits.items():
        print(f"\nFold {fold_num}:")
        
        # Training set analysis
        train_df = df[df['dataset_id'].isin(fold_data['train'])]
        train_covid = train_df['COVID-19'].sum()
        train_normal = train_df['normal'].sum()
        train_total = train_df['total_samples'].sum()
        
        print(f"  Training set:")
        print(f"    Datasets: {len(fold_data['train'])}")
        print(f"    COVID-19 samples: {train_covid:,} ({train_covid/train_total*100:.1f}%)")
        print(f"    Normal samples: {train_normal:,} ({train_normal/train_total*100:.1f}%)")
        print(f"    Total samples: {train_total:,}")
        
        # Validation set analysis
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        eval_covid = eval_df['COVID-19'].sum()
        eval_normal = eval_df['normal'].sum()
        eval_total = eval_df['total_samples'].sum()
        
        print(f"  Evaluation set:")
        print(f"    Datasets: {len(fold_data['eval'])}")
        print(f"    COVID-19 samples: {eval_covid:,} ({eval_covid/eval_total*100:.1f}%)")
        print(f"    Normal samples: {eval_normal:,} ({eval_normal/eval_total*100:.1f}%)")
        print(f"    Total samples: {eval_total:,}")
        
        # List dataset IDs in eval set
        print(f"    Eval dataset IDs: {fold_data['eval']}")
    
    # Data leakage check
    print(f"\n" + "=" * 80)
    print("DATA LEAKAGE CHECK")
    print("=" * 80)
    
    all_train_ids = set()
    all_eval_ids = set()
    leakage_found = False
    
    for fold_num, fold_data in cv_splits.items():
        train_ids = set(fold_data['train'])
        eval_ids = set(fold_data['eval'])
        
        # Check for overlap within the same fold
        overlap = train_ids.intersection(eval_ids)
        if overlap:
            print(f"❌ LEAKAGE DETECTED in Fold {fold_num}: {len(overlap)} datasets in both train and eval")
            print(f"   Overlapping dataset IDs: {list(overlap)}")
            leakage_found = True
        else:
            print(f"✅ Fold {fold_num}: No overlap between train and eval sets")
        
        all_train_ids.update(train_ids)
        all_eval_ids.update(eval_ids)
    
    # Check if any dataset appears in multiple eval sets
    from collections import Counter
    eval_counts = Counter()
    for fold_num, fold_data in cv_splits.items():
        for dataset_id in fold_data['eval']:
            eval_counts[dataset_id] += 1
    
    multi_eval = {dataset_id: count for dataset_id, count in eval_counts.items() if count > 1}
    if multi_eval:
        print(f"❌ LEAKAGE DETECTED: {len(multi_eval)} datasets appear in multiple eval sets")
        for dataset_id, count in multi_eval.items():
            print(f"   Dataset {dataset_id} appears in {count} eval sets")
        leakage_found = True
    
    # Verify all datasets are included exactly once in evaluation
    all_dataset_ids = set(df['dataset_id'].tolist())
    if all_eval_ids == all_dataset_ids:
        print(f"✅ All {len(all_dataset_ids)} datasets appear exactly once in evaluation sets")
    else:
        missing = all_dataset_ids - all_eval_ids
        extra = all_eval_ids - all_dataset_ids
        if missing:
            print(f"❌ Missing datasets in eval sets: {missing}")
        if extra:
            print(f"❌ Extra datasets in eval sets: {extra}")
        leakage_found = True
    
    if not leakage_found:
        print(f"🎉 NO DATA LEAKAGE DETECTED: All splits are properly separated!")
    
    print(f"\nSummary:")
    print(f"  Total unique datasets in training across all folds: {len(all_train_ids)}")
    print(f"  Total unique datasets in evaluation across all folds: {len(all_eval_ids)}")
    print(f"  Expected datasets in training per fold: {len(all_dataset_ids) * 4 // 5}")
    print(f"  Expected datasets in evaluation per fold: {len(all_dataset_ids) // 5}")
    
    print(f"\n✅ Cross-validation splits created successfully!")

except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import numpy as np

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# Paths
csv_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/dataset_summary.csv"
json_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/5fold_cv_splits.json"

print("Loading CellNexus COVID dataset summary and CV splits...")
try:
    # Load the CSV data
    df = pd.read_csv(csv_path)
    print(f"Dataset summary loaded successfully! Shape: {df.shape}")
    
    # Load JSON splits
    print("Loading cross-validation splits...")
    with open(json_path, 'r') as f:
        cv_splits = json.load(f)
    
    print(f"Loaded {len(cv_splits)} folds")
    
    # COMPREHENSIVE DATA LEAKAGE CHECK
    print("\n" + "="*80)
    print("COMPREHENSIVE DATA LEAKAGE CHECK")
    print("="*80)
    
    all_train_ids = set()
    all_eval_ids = set()
    leakage_found = False
    
    # Check 1: Within-fold overlap
    print("Check 1: Within-fold train/eval overlap")
    for fold_num, fold_data in cv_splits.items():
        train_ids = set(fold_data['train'])
        eval_ids = set(fold_data['eval'])
        
        overlap = train_ids.intersection(eval_ids)
        if overlap:
            print(f"❌ LEAKAGE in Fold {fold_num}: {len(overlap)} datasets in both train and eval")
            print(f"   Overlapping IDs: {list(overlap)}")
            leakage_found = True
        else:
            print(f"✅ Fold {fold_num}: No train/eval overlap ({len(train_ids)} train, {len(eval_ids)} eval)")
        
        all_train_ids.update(train_ids)
        all_eval_ids.update(eval_ids)
    
    # Check 2: Cross-fold eval overlap
    print(f"\nCheck 2: Cross-fold evaluation overlap")
    eval_counts = Counter()
    for fold_num, fold_data in cv_splits.items():
        for dataset_id in fold_data['eval']:
            eval_counts[dataset_id] += 1
    
    multi_eval = {dataset_id: count for dataset_id, count in eval_counts.items() if count > 1}
    if multi_eval:
        print(f"❌ LEAKAGE: {len(multi_eval)} datasets in multiple eval sets")
        for dataset_id, count in multi_eval.items():
            print(f"   {dataset_id}: appears in {count} eval sets")
        leakage_found = True
    else:
        print(f"✅ No dataset appears in multiple eval sets")
    
    # Check 3: Complete dataset coverage
    print(f"\nCheck 3: Dataset coverage verification")
    all_dataset_ids = set(df['dataset_id'].tolist())
    
    if all_eval_ids == all_dataset_ids:
        print(f"✅ Perfect coverage: All {len(all_dataset_ids)} datasets appear exactly once in eval")
    else:
        missing = all_dataset_ids - all_eval_ids
        extra = all_eval_ids - all_dataset_ids
        if missing:
            print(f"❌ Missing from eval: {missing}")
        if extra:
            print(f"❌ Extra in eval: {extra}")
        leakage_found = True
    
    # Check 4: Fold size consistency
    print(f"\nCheck 4: Fold size consistency")
    expected_eval_size = len(all_dataset_ids) // 5
    eval_sizes = [len(fold_data['eval']) for fold_data in cv_splits.values()]
    
    if all(size in [expected_eval_size, expected_eval_size + 1] for size in eval_sizes):
        print(f"✅ Consistent fold sizes: {eval_sizes} (expected ~{expected_eval_size})")
    else:
        print(f"⚠️  Inconsistent fold sizes: {eval_sizes}")
    
    # Final leakage verdict
    print(f"\n{'='*20} LEAKAGE CHECK RESULT {'='*20}")
    if not leakage_found:
        print("🎉 NO DATA LEAKAGE DETECTED - All splits are valid!")
    else:
        print("❌ DATA LEAKAGE FOUND - Splits need to be fixed!")
    print("="*64)
    
    # CREATE COMPREHENSIVE VISUALIZATION
    print(f"\n" + "="*80)
    print("CREATING COMPREHENSIVE VISUALIZATION")
    print("="*80)
    
    # Create figure with subplots
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    fig.suptitle('CellNexus COVID 5-Fold Cross-Validation Analysis', fontsize=16, fontweight='bold')
    
    # Colors for consistency
    colors = ['#FF6B6B', '#4ECDC4']  # Red for COVID, Teal for Normal
    disease_order = ['COVID-19', 'normal']
    
    # Plot 1: Sample distribution across folds
    ax1 = axes[0, 0]
    fold_stats = []
    
    for fold_num, fold_data in cv_splits.items():
        train_df = df[df['dataset_id'].isin(fold_data['train'])]
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        
        train_covid = train_df['COVID-19'].sum()
        train_normal = train_df['normal'].sum()
        eval_covid = eval_df['COVID-19'].sum()
        eval_normal = eval_df['normal'].sum()
        
        fold_stats.append({
            'fold': f'Fold {fold_num}',
            'train_covid': train_covid,
            'train_normal': train_normal,
            'eval_covid': eval_covid,
            'eval_normal': eval_normal
        })
    
    fold_df = pd.DataFrame(fold_stats)
    
    # Stacked bar for training and evaluation
    x = np.arange(len(fold_df))
    width = 0.35
    
    # Training bars
    ax1.bar(x - width/2, fold_df['train_covid'], width, label='Train COVID-19', 
           color=colors[0], alpha=0.8)
    ax1.bar(x - width/2, fold_df['train_normal'], width, bottom=fold_df['train_covid'],
           label='Train Normal', color=colors[1], alpha=0.8)
    
    # Evaluation bars
    ax1.bar(x + width/2, fold_df['eval_covid'], width, label='Eval COVID-19', 
           color=colors[0], alpha=0.5)
    ax1.bar(x + width/2, fold_df['eval_normal'], width, bottom=fold_df['eval_covid'],
           label='Eval Normal', color=colors[1], alpha=0.5)
    
    ax1.set_title('Sample Distribution Across Folds', fontweight='bold')
    ax1.set_xlabel('Fold')
    ax1.set_ylabel('Number of Samples')
    ax1.set_xticks(x)
    ax1.set_xticklabels(fold_df['fold'])
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Plot 2: Dataset distribution across folds
    ax2 = axes[0, 1]
    dataset_counts = []
    
    for fold_num, fold_data in cv_splits.items():
        train_datasets = len(fold_data['train'])
        eval_datasets = len(fold_data['eval'])
        dataset_counts.append({
            'fold': f'Fold {fold_num}',
            'train': train_datasets,
            'eval': eval_datasets
        })
    
    dataset_df = pd.DataFrame(dataset_counts)
    
    x = np.arange(len(dataset_df))
    ax2.bar(x - width/2, dataset_df['train'], width, label='Train Datasets', alpha=0.8)
    ax2.bar(x + width/2, dataset_df['eval'], width, label='Eval Datasets', alpha=0.8)
    
    ax2.set_title('Dataset Count Distribution Across Folds', fontweight='bold')
    ax2.set_xlabel('Fold')
    ax2.set_ylabel('Number of Datasets')
    ax2.set_xticks(x)
    ax2.set_xticklabels(dataset_df['fold'])
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    # Plot 3: Disease balance in evaluation sets
    ax3 = axes[1, 0]
    eval_balance = []
    
    for fold_num, fold_data in cv_splits.items():
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        covid_samples = eval_df['COVID-19'].sum()
        normal_samples = eval_df['normal'].sum()
        total_samples = covid_samples + normal_samples
        
        eval_balance.append({
            'fold': f'Fold {fold_num}',
            'COVID-19': covid_samples,
            'normal': normal_samples,
            'covid_pct': covid_samples / total_samples * 100 if total_samples > 0 else 0,
            'normal_pct': normal_samples / total_samples * 100 if total_samples > 0 else 0
        })
    
    balance_df = pd.DataFrame(eval_balance)
    
    x = np.arange(len(balance_df))
    ax3.bar(x, balance_df['COVID-19'], label='COVID-19', color=colors[0], alpha=0.8)
    ax3.bar(x, balance_df['normal'], bottom=balance_df['COVID-19'], 
           label='Normal', color=colors[1], alpha=0.8)
    
    # Add percentage labels
    for i, (covid, normal) in enumerate(zip(covid_counts, normal_counts)):
        total = covid + normal
        covid_pct = covid / total * 100 if total > 0 else 0
        normal_pct = normal / total * 100 if total > 0 else 0
        
        ax5.text(i, covid/2, f"{covid_pct:.1f}%", ha='center', va='center', 
                fontweight='bold', color='white')
        ax5.text(i, covid + normal/2, f"{normal_pct:.1f}%", ha='center', va='center', 
                fontweight='bold', color='white')
    
    ax5.set_title('Overall Train vs Eval Distribution', fontweight='bold')
    ax5.set_xlabel('Split Type')
    ax5.set_ylabel('Number of Samples')
    ax5.set_xticks(x)
    ax5.set_xticklabels(categories)
    ax5.legend()
    ax5.grid(axis='y', alpha=0.3)
    
    # Plot 6: Fold balance metrics
    ax6 = axes[2, 1]
    
    # Calculate balance metrics for each fold
    balance_metrics = []
    for i, row in balance_df.iterrows():
        # Calculate deviation from 50-50 balance
        covid_dev = abs(row['covid_pct'] - 50)
        balance_score = 100 - covid_dev  # Higher is more balanced
        balance_metrics.append(balance_score)
    
    bars = ax6.bar(range(len(balance_metrics)), balance_metrics, 
                   color=['green' if score > 80 else 'orange' if score > 60 else 'red' 
                         for score in balance_metrics], alpha=0.7)
    
    # Add value labels on bars
    for i, (bar, score) in enumerate(zip(bars, balance_metrics)):
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{score:.1f}', ha='center', va='bottom', fontweight='bold')
    
    ax6.set_title('Fold Balance Score (Higher = More Balanced)', fontweight='bold')
    ax6.set_xlabel('Fold')
    ax6.set_ylabel('Balance Score')
    ax6.set_xticks(range(len(balance_metrics)))
    ax6.set_xticklabels([f'Fold {i+1}' for i in range(len(balance_metrics))])
    ax6.set_ylim(0, 110)
    ax6.grid(axis='y', alpha=0.3)
    
    # Add color legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='green', alpha=0.7, label='Excellent (>80)'),
                      Patch(facecolor='orange', alpha=0.7, label='Good (60-80)'),
                      Patch(facecolor='red', alpha=0.7, label='Poor (<60)')]
    ax6.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    
    # Save the plot
    plot_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/cv_splits_analysis.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"Comprehensive visualization saved to: {plot_path}")
    
    plt.show()
    
    # DETAILED STATISTICAL ANALYSIS
    print(f"\n" + "="*80)
    print("DETAILED STATISTICAL ANALYSIS")
    print("="*80)
    
    print(f"\nOverall Dataset Statistics:")
    print(f"  Total datasets: {len(df)}")
    print(f"  Total samples: {df['total_samples'].sum():,}")
    print(f"  COVID-19 samples: {df['COVID-19'].sum():,} ({df['COVID-19'].sum()/df['total_samples'].sum()*100:.1f}%)")
    print(f"  Normal samples: {df['normal'].sum():,} ({df['normal'].sum()/df['total_samples'].sum()*100:.1f}%)")
    
    print(f"\nFold-wise Detailed Analysis:")
    for fold_num, fold_data in cv_splits.items():
        print(f"\n--- Fold {fold_num} ---")
        
        # Training set
        train_df = df[df['dataset_id'].isin(fold_data['train'])]
        train_covid = train_df['COVID-19'].sum()
        train_normal = train_df['normal'].sum()
        train_total = train_covid + train_normal
        
        print(f"Training Set ({len(fold_data['train'])} datasets):")
        print(f"  COVID-19: {train_covid:,} samples ({train_covid/train_total*100:.1f}%)")
        print(f"  Normal: {train_normal:,} samples ({train_normal/train_total*100:.1f}%)")
        print(f"  Total: {train_total:,} samples")
        
        # Evaluation set
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        eval_covid = eval_df['COVID-19'].sum()
        eval_normal = eval_df['normal'].sum()
        eval_total = eval_covid + eval_normal
        
        print(f"Evaluation Set ({len(fold_data['eval'])} datasets):")
        print(f"  COVID-19: {eval_covid:,} samples ({eval_covid/eval_total*100:.1f}%)")
        print(f"  Normal: {eval_normal:,} samples ({eval_normal/eval_total*100:.1f}%)")
        print(f"  Total: {eval_total:,} samples")
        
        # Balance metrics
        covid_balance_diff = abs((eval_covid/eval_total*100) - (train_covid/train_total*100))
        print(f"  Balance difference: {covid_balance_diff:.1f}% (COVID-19 % diff between train/eval)")
        
        # Dataset composition
        print(f"  Eval dataset details:")
        for dataset_id in fold_data['eval']:
            row = df[df['dataset_id'] == dataset_id].iloc[0]
            covid_count = row['COVID-19']
            normal_count = row['normal']
            total_count = row['total_samples']
            dominant = 'COVID-19' if covid_count > normal_count else 'Normal' if normal_count > covid_count else 'Mixed'
            print(f"    {dataset_id}: {covid_count:,} COVID + {normal_count:,} normal = {total_count:,} total ({dominant})")
    
    # Cross-validation quality metrics
    print(f"\n" + "="*80)
    print("CROSS-VALIDATION QUALITY METRICS")
    print("="*80)
    
    # Calculate variance in fold sizes
    eval_sizes = [len(cv_splits[str(i+1)]['eval']) for i in range(5)]
    train_sizes = [len(cv_splits[str(i+1)]['train']) for i in range(5)]
    
    print(f"\nFold Size Consistency:")
    print(f"  Eval fold sizes: {eval_sizes}")
    print(f"  Train fold sizes: {train_sizes}")
    print(f"  Eval size variance: {np.var(eval_sizes):.2f}")
    print(f"  Train size variance: {np.var(train_sizes):.2f}")
    
    # Calculate balance consistency across folds
    covid_percentages = [row['covid_pct'] for _, row in balance_df.iterrows()]
    balance_variance = np.var(covid_percentages)
    
    print(f"\nDisease Balance Consistency:")
    print(f"  COVID-19 percentages across eval folds: {[f'{p:.1f}%' for p in covid_percentages]}")
    print(f"  Balance variance: {balance_variance:.2f}")
    print(f"  Balance standard deviation: {np.std(covid_percentages):.1f}%")
    
    # Overall quality assessment
    print(f"\n" + "="*40)
    print("OVERALL QUALITY ASSESSMENT")
    print("="*40)
    
    quality_score = 0
    max_score = 100
    
    # Leakage check (40 points)
    if not leakage_found:
        quality_score += 40
        print("✅ No data leakage: +40 points")
    else:
        print("❌ Data leakage detected: +0 points")
    
    # Fold size consistency (20 points)
    if np.var(eval_sizes) == 0:
        quality_score += 20
        print("✅ Perfect fold size consistency: +20 points")
    elif np.var(eval_sizes) <= 1:
        quality_score += 15
        print("✅ Good fold size consistency: +15 points")
    else:
        quality_score += 5
        print("⚠️ Poor fold size consistency: +5 points")
    
    # Disease balance (25 points)
    if balance_variance < 25:  # Less than 5% std dev
        quality_score += 25
        print("✅ Excellent disease balance: +25 points")
    elif balance_variance < 100:  # Less than 10% std dev
        quality_score += 20
        print("✅ Good disease balance: +20 points")
    elif balance_variance < 225:  # Less than 15% std dev
        quality_score += 15
        print("⚠️ Fair disease balance: +15 points")
    else:
        quality_score += 5
        print("⚠️ Poor disease balance: +5 points")
    
    # Coverage completeness (15 points)
    if len(all_eval_ids) == len(df):
        quality_score += 15
        print("✅ Complete dataset coverage: +15 points")
    else:
        quality_score += 0
        print("❌ Incomplete dataset coverage: +0 points")
    
    print(f"\n🏆 FINAL QUALITY SCORE: {quality_score}/{max_score} ({quality_score/max_score*100:.1f}%)")
    
    if quality_score >= 90:
        print("🌟 EXCELLENT: Cross-validation splits are high quality!")
    elif quality_score >= 70:
        print("👍 GOOD: Cross-validation splits are acceptable with minor issues")
    elif quality_score >= 50:
        print("⚠️ FAIR: Cross-validation splits have some issues that should be addressed")
    else:
        print("❌ POOR: Cross-validation splits have significant issues and should be recreated")
    
    # Export detailed results
    print(f"\n" + "="*80)
    print("EXPORTING RESULTS")
    print("="*80)
    
    # Save detailed fold statistics
    fold_stats_df = pd.DataFrame(fold_stats)
    stats_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/fold_statistics.csv"
    fold_stats_df.to_csv(stats_path, index=False)
    print(f"Fold statistics exported to: {stats_path}")
    
    # Save quality report
    quality_report = {
        'total_datasets': len(df),
        'total_samples': int(df['total_samples'].sum()),
        'covid_samples': int(df['COVID-19'].sum()),
        'normal_samples': int(df['normal'].sum()),
        'leakage_detected': leakage_found,
        'fold_size_variance': float(np.var(eval_sizes)),
        'balance_variance': float(balance_variance),
        'quality_score': quality_score,
        'covid_percentages_by_fold': covid_percentages
    }
    
    report_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/quality_report.json"
    with open(report_path, 'w') as f:
        json.dump(quality_report, f, indent=2)
    print(f"Quality report exported to: {report_path}")
    
    print(f"\n🎉 Analysis completed successfully!")
    print(f"📊 Visualization: {plot_path}")
    print(f"📈 Statistics: {stats_path}")
    print(f"📋 Quality Report: {report_path}")

except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()
    for i, row in balance_df.iterrows():
        total = row['COVID-19'] + row['normal']
        ax3.text(i, total/2, f"{row['covid_pct']:.1f}%", ha='center', va='center', 
                fontweight='bold', color='white')
        ax3.text(i, row['COVID-19'] + row['normal']/2, f"{row['normal_pct']:.1f}%", 
                ha='center', va='center', fontweight='bold', color='white')
    
    ax3.set_title('Disease Balance in Evaluation Sets', fontweight='bold')
    ax3.set_xlabel('Fold')
    ax3.set_ylabel('Number of Samples')
    ax3.set_xticks(x)
    ax3.set_xticklabels(balance_df['fold'])
    ax3.legend()
    ax3.grid(axis='y', alpha=0.3)
    
    # Plot 4: Dataset size distribution in eval sets
    ax4 = axes[1, 1]
    all_eval_sizes = []
    fold_labels = []
    
    for fold_num, fold_data in cv_splits.items():
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        sizes = eval_df['total_samples'].tolist()
        all_eval_sizes.extend(sizes)
        fold_labels.extend([f'Fold {fold_num}'] * len(sizes))
    
    size_data = pd.DataFrame({'fold': fold_labels, 'size': all_eval_sizes})
    sns.boxplot(data=size_data, x='fold', y='size', ax=ax4)
    ax4.set_title('Dataset Size Distribution in Eval Sets', fontweight='bold')
    ax4.set_xlabel('Fold')
    ax4.set_ylabel('Dataset Size (log scale)')
    ax4.set_yscale('log')
    
    # Plot 5: Overall statistics comparison
    ax5 = axes[2, 0]
    
    # Calculate overall train/eval statistics
    overall_stats = {
        'train_covid': sum(fold_df['train_covid']),
        'train_normal': sum(fold_df['train_normal']),
        'eval_covid': sum(fold_df['eval_covid']),
        'eval_normal': sum(fold_df['eval_normal'])
    }
    
    categories = ['Train', 'Eval']
    covid_counts = [overall_stats['train_covid'], overall_stats['eval_covid']]
    normal_counts = [overall_stats['train_normal'], overall_stats['eval_normal']]
    
    x = np.arange(len(categories))
    ax5.bar(x, covid_counts, label='COVID-19', color=colors[0], alpha=0.8)
    ax5.bar(x, normal_counts, bottom=covid_counts, label='Normal', color=colors[1], alpha=0.8)
    
    # Add percentage labels

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from collections import Counter
import numpy as np

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# Paths - adapted for COVID dataset
dataset_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/tokenized_30M/cellnexus_singlecell.dataset/"
json_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/5fold_cv_splits.json"

print("Loading HuggingFace COVID dataset...")
try:
    # Load the dataset
    dataset = Dataset.load_from_disk(dataset_path)
    df = dataset.to_pandas()
    print(f"Dataset loaded successfully! Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Load JSON splits
    print("\nLoading cross-validation splits...")
    with open(json_path, 'r') as f:
        cv_splits = json.load(f)
    
    print(f"Loaded {len(cv_splits)} folds")
    
    # Check for data leakage across all folds
    print("\n" + "="*60)
    print("CHECKING FOR DATA LEAKAGE")
    print("="*60)
    
    all_train_ids = set()
    all_eval_ids = set()
    leakage_found = False
    
    for fold_num, fold_data in cv_splits.items():
        train_ids = set(fold_data['train'])
        eval_ids = set(fold_data['eval'])
        
        # Check for overlap within the same fold
        overlap = train_ids.intersection(eval_ids)
        if overlap:
            print(f"❌ LEAKAGE DETECTED in Fold {fold_num}: {len(overlap)} datasets in both train and eval")
            print(f"   Overlapping IDs: {list(overlap)}")
            leakage_found = True
        else:
            print(f"✅ Fold {fold_num}: No overlap between train and eval sets")
        
        all_train_ids.update(train_ids)
        all_eval_ids.update(eval_ids)
    
    # Check if any dataset appears in multiple eval sets
    eval_counts = Counter()
    for fold_num, fold_data in cv_splits.items():
        for dataset_id in fold_data['eval']:
            eval_counts[dataset_id] += 1
    
    multi_eval = {dataset_id: count for dataset_id, count in eval_counts.items() if count > 1}
    if multi_eval:
        print(f"❌ LEAKAGE DETECTED: {len(multi_eval)} datasets appear in multiple eval sets")
        for dataset_id, count in multi_eval.items():
            print(f"   Dataset {dataset_id} appears in {count} eval sets")
        leakage_found = True
    
    if not leakage_found:
        print("✅ NO DATA LEAKAGE DETECTED: All splits are properly separated")
    
    print(f"\nTotal unique datasets in training across all folds: {len(all_train_ids)}")
    print(f"Total unique datasets in evaluation across all folds: {len(all_eval_ids)}")
    
    # Create visualization
    print("\n" + "="*60)
    print("CREATING VISUALIZATION")
    print("="*60)
    
    # Create subplots: 2 rows, 3 columns (5 folds + 1 overall)
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('COVID Dataset: Class Distribution Across 5-Fold Cross-Validation Splits', fontsize=16, fontweight='bold')
    
    # Flatten axes for easier indexing
    axes = axes.flatten()
    
    # Define colors for consistency - adapted for 2 diseases
    colors = ['#FF6B6B', '#4ECDC4']  # Red for COVID-19, Teal for normal
    disease_order = ['COVID-19', 'normal']
    
    # Plot each fold
    for i, (fold_num, fold_data) in enumerate(cv_splits.items()):
        ax = axes[i]
        
        # Get data for this fold - using dataset_id instead of individual
        train_df = df[df['dataset_id'].isin(fold_data['train'])]
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        
        # Count samples by disease
        train_counts = train_df['disease'].value_counts()
        eval_counts = eval_df['disease'].value_counts()
        
        # Ensure all diseases are represented (even if count is 0)
        train_counts = train_counts.reindex(disease_order, fill_value=0)
        eval_counts = eval_counts.reindex(disease_order, fill_value=0)
        
        # Create bar plot
        x = np.arange(len(disease_order))
        width = 0.35
        
        bars1 = ax.bar(x - width/2, train_counts.values, width, label='Train', 
                      color=colors, alpha=0.8)
        bars2 = ax.bar(x + width/2, eval_counts.values, width, label='Eval', 
                      color=colors, alpha=0.5)
        
        # Add value labels on bars
        for bar in bars1:
            height = bar.get_height()
            if height > 0:  # Only show label if there are samples
                ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height):,}', ha='center', va='bottom', fontweight='bold')
        
        for bar in bars2:
            height = bar.get_height()
            if height > 0:  # Only show label if there are samples
                ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height):,}', ha='center', va='bottom', fontweight='bold')
        
        # Customize subplot
        ax.set_title(f'Fold {fold_num}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Disease Type', fontsize=10)
        ax.set_ylabel('Number of Samples', fontsize=10)
        ax.set_xticks(x)
        ax.set_xticklabels(disease_order, fontsize=10)
        ax.legend(fontsize=9)
        ax.grid(axis='y', alpha=0.3)
        
        # Add fold statistics as text - using dataset_id terminology
        train_total = len(train_df)
        eval_total = len(eval_df)
        train_datasets = len(fold_data['train'])
        eval_datasets = len(fold_data['eval'])
        
        stats_text = f'Train: {train_total:,} samples ({train_datasets} datasets)\n'
        stats_text += f'Eval: {eval_total:,} samples ({eval_datasets} datasets)'
        
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                fontsize=8)
    
    # Create overall statistics plot in the last subplot
    ax_overall = axes[5]
    
    # Calculate overall statistics
    overall_train_counts = df['disease'].value_counts().reindex(disease_order, fill_value=0)
    
    # Calculate average eval counts across folds
    avg_eval_counts = []
    for disease in disease_order:
        total_eval = sum(df[df['dataset_id'].isin(fold_data['eval'])]['disease'].value_counts().get(disease, 0) 
                        for fold_data in cv_splits.values())
        avg_eval_counts.append(total_eval / len(cv_splits))
    
    # Plot overall statistics
    x = np.arange(len(disease_order))
    bars1 = ax_overall.bar(x - width/2, overall_train_counts.values, width, 
                          label='Total Dataset', color=colors, alpha=0.8)
    bars2 = ax_overall.bar(x + width/2, avg_eval_counts, width, 
                          label='Avg Eval per Fold', color=colors, alpha=0.5)
    
    # Add value labels
    for bar in bars1:
        height = bar.get_height()
        ax_overall.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height):,}', ha='center', va='bottom', fontweight='bold')
    
    for bar in bars2:
        height = bar.get_height()
        ax_overall.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height):,}', ha='center', va='bottom', fontweight='bold')
    
    ax_overall.set_title('Overall Dataset Statistics', fontsize=12, fontweight='bold')
    ax_overall.set_xlabel('Disease Type', fontsize=10)
    ax_overall.set_ylabel('Number of Samples', fontsize=10)
    ax_overall.set_xticks(x)
    ax_overall.set_xticklabels(disease_order, fontsize=10)
    ax_overall.legend(fontsize=9)
    ax_overall.grid(axis='y', alpha=0.3)
    
    # Add overall statistics text
    total_samples = len(df)
    total_datasets = df['dataset_id'].nunique()
    stats_text = f'Total: {total_samples:,} samples\n'
    stats_text += f'Datasets: {total_datasets}\n'
    stats_text += f'Avg per fold: {total_samples//5:,} samples'
    
    ax_overall.text(0.02, 0.98, stats_text, transform=ax_overall.transAxes, 
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                   fontsize=8)
    
    plt.tight_layout()
    
    # Save the plot
    plot_path = "/hpcfs/groups/phoenix-hpc-mangiola_laboratory/haroon/geneformer/datasets/disease_classification/cellnexus_covid_disease/cv_splits_visualization.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"Visualization saved to: {plot_path}")
    
    plt.show()
    
    # Print detailed statistics
    print("\n" + "="*60)
    print("DETAILED FOLD STATISTICS")
    print("="*60)
    
    for fold_num, fold_data in cv_splits.items():
        print(f"\nFold {fold_num}:")
        
        train_df = df[df['dataset_id'].isin(fold_data['train'])]
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        
        print(f"  Training:")
        print(f"    Datasets: {len(fold_data['train'])}")
        print(f"    Samples: {len(train_df):,}")
        train_dist = train_df['disease'].value_counts()
        for disease in disease_order:
            count = train_dist.get(disease, 0)
            pct = (count / len(train_df)) * 100 if len(train_df) > 0 else 0
            print(f"    {disease}: {count:,} ({pct:.1f}%)")
        
        print(f"  Evaluation:")
        print(f"    Datasets: {len(fold_data['eval'])}")
        print(f"    Samples: {len(eval_df):,}")
        eval_dist = eval_df['disease'].value_counts()
        for disease in disease_order:
            count = eval_dist.get(disease, 0)
            pct = (count / len(eval_df)) * 100 if len(eval_df) > 0 else 0
            print(f"    {disease}: {count:,} ({pct:.1f}%)")
        
        # Show dataset composition in eval set
        print(f"  Evaluation dataset breakdown:")
        for dataset_id in sorted(fold_data['eval']):
            dataset_samples = df[df['dataset_id'] == dataset_id]
            dataset_dist = dataset_samples['disease'].value_counts()
            total_dataset_samples = len(dataset_samples)
            covid_count = dataset_dist.get('COVID-19', 0)
            normal_count = dataset_dist.get('normal', 0)
            dominant = 'COVID-19' if covid_count > normal_count else 'normal' if normal_count > covid_count else 'mixed'
            print(f"    {dataset_id}: {total_dataset_samples:,} samples ({covid_count:,} COVID, {normal_count:,} normal) - {dominant}")
    
    # Additional COVID-specific analysis
    print(f"\n" + "="*60)
    print("COVID-SPECIFIC ANALYSIS")
    print("="*60)
    
    # Analyze dataset types across folds
    print(f"\nDataset type distribution across folds:")
    for fold_num, fold_data in cv_splits.items():
        eval_datasets = fold_data['eval']
        covid_only = 0
        normal_only = 0
        mixed = 0
        
        for dataset_id in eval_datasets:
            dataset_samples = df[df['dataset_id'] == dataset_id]
            covid_count = len(dataset_samples[dataset_samples['disease'] == 'COVID-19'])
            normal_count = len(dataset_samples[dataset_samples['disease'] == 'normal'])
            
            if covid_count > 0 and normal_count == 0:
                covid_only += 1
            elif normal_count > 0 and covid_count == 0:
                normal_only += 1
            elif covid_count > 0 and normal_count > 0:
                mixed += 1
        
        print(f"  Fold {fold_num}: {covid_only} COVID-only, {normal_only} normal-only, {mixed} mixed datasets")
    
    # Disease balance analysis
    print(f"\nDisease balance across evaluation folds:")
    for fold_num, fold_data in cv_splits.items():
        eval_df = df[df['dataset_id'].isin(fold_data['eval'])]
        covid_samples = len(eval_df[eval_df['disease'] == 'COVID-19'])
        normal_samples = len(eval_df[eval_df['disease'] == 'normal'])
        total_samples = covid_samples + normal_samples
        
        covid_pct = (covid_samples / total_samples * 100) if total_samples > 0 else 0
        normal_pct = (normal_samples / total_samples * 100) if total_samples > 0 else 0
        balance_score = 100 - abs(covid_pct - 50)  # Higher score = more balanced
        
        print(f"  Fold {fold_num}: {covid_pct:.1f}% COVID-19, {normal_pct:.1f}% normal (Balance score: {balance_score:.1f})")
    
    print(f"\n✅ Analysis completed successfully!")
    print(f"📊 Visualization saved to: {plot_path}")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()